In [14]:
import pandas as pd
import pickle
import csv
import random
from tqdm import tqdm_notebook as tqdm
from collections import Counter, defaultdict
from pandas.tseries.offsets import DateOffset
from scipy.stats import pearsonr
import numpy as np
import re
import os

from IPython.core.debugger import Tracer; debug_here = Tracer()

# Calculate revert-corrected sum diff outcome measure

In [3]:
samp = pd.read_csv('/home/michael/school/research/wp/enwiki_qualitative_analysis.csv', parse_dates=['timestamp'])
talk = samp
# talk = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/all_talk.csv', 
#                    names=['id', 'article', 'thread', 'timestamp', 'editor', 'thread_type', 'text'],
#                   parse_dates=['timestamp'])
# talk.drop(['id'], axis=1, inplace=True)

keys = set(zip(talk['article'], talk['thread']))
talk

,article,thread,editor,timestamp,thread_starter,text,editor_score
0,2005 New Orleans flood,Conspiracy Theories,Malleus Fatuorum,2008-05-30 12:44:13,THREAD_STARTER,While I don't buy the idea that the government...,0.490798
1,2005 New Orleans flood,Conspiracy Theories,Don\'tKnowItAtAll,2011-01-01 21:11:56,POST,"A bit late to reply to this subject, but Katri...",0.747573
2,2005 New Orleans flood,Conspiracy Theories,Sedna1000,2011-04-09 23:16:44,POST,Why is this even on Wikipedia? The flooding of...,1.000000
3,2005 New Orleans flood,Conspiracy Theories,Infrogmation,2011-04-10 12:36:53,POST,It's on Wikipedia because it is a rumor or fol...,0.602496
4,2005 New Orleans flood,Conspiracy Theories,Sedna1000,2011-04-13 01:54:18,POST,"Well maybe the article should read ""although b...",1.000000
5,2007 French riots,Wikipedia routinely lists the ethnic make up o...,Mnw2000,2007-11-27 15:57:40,THREAD_STARTER,Take any simple article about an individual an...,NaN
6,2007 French riots,Wikipedia routinely lists the ethnic make up o...,Rama,2007-11-27 15:59:35,POST,"I you are referring to the victims, as for now...",0.854220
7,2007 French riots,Wikipedia routinely lists the ethnic make up o...,Meithal,2007-11-27 16:03:45,POST,: Rame : see http://jt.france2.fr/20h/ Meithal...,NaN
8,2007 French riots,Wikipedia routinely lists the ethnic make up o...,RenniePet,2007-11-27 16:59:12,POST,"::The BBC article says, at the very bottom, th...",0.857143
9,2007 French riots,Wikipedia routinely lists the ethnic make up o...,Rama,2007-11-27 17:00:46,POST,::: Does that mean that they are French ? Rama...,0.854220


In [4]:
# Filter all_talk
conv_sizes = talk.groupby(['article', 'thread']).size()
conv_sizes.name = 'conv_size'
talk = talk.join(conv_sizes, on=['article', 'thread'])
talk.sort_values(['article', 'thread'], inplace=True)

print(len(talk))
talk = talk[talk['conv_size']>=2]
print(len(talk))
talk

514
514


,article,thread,editor,timestamp,thread_starter,text,editor_score,conv_size
0,2005 New Orleans flood,Conspiracy Theories,Malleus Fatuorum,2008-05-30 12:44:13,THREAD_STARTER,While I don't buy the idea that the government...,0.490798,5
1,2005 New Orleans flood,Conspiracy Theories,Don\'tKnowItAtAll,2011-01-01 21:11:56,POST,"A bit late to reply to this subject, but Katri...",0.747573,5
2,2005 New Orleans flood,Conspiracy Theories,Sedna1000,2011-04-09 23:16:44,POST,Why is this even on Wikipedia? The flooding of...,1.000000,5
3,2005 New Orleans flood,Conspiracy Theories,Infrogmation,2011-04-10 12:36:53,POST,It's on Wikipedia because it is a rumor or fol...,0.602496,5
4,2005 New Orleans flood,Conspiracy Theories,Sedna1000,2011-04-13 01:54:18,POST,"Well maybe the article should read ""although b...",1.000000,5
5,2007 French riots,Wikipedia routinely lists the ethnic make up o...,Mnw2000,2007-11-27 15:57:40,THREAD_STARTER,Take any simple article about an individual an...,NaN,12
6,2007 French riots,Wikipedia routinely lists the ethnic make up o...,Rama,2007-11-27 15:59:35,POST,"I you are referring to the victims, as for now...",0.854220,12
7,2007 French riots,Wikipedia routinely lists the ethnic make up o...,Meithal,2007-11-27 16:03:45,POST,: Rame : see http://jt.france2.fr/20h/ Meithal...,NaN,12
8,2007 French riots,Wikipedia routinely lists the ethnic make up o...,RenniePet,2007-11-27 16:59:12,POST,"::The BBC article says, at the very bottom, th...",0.857143,12
9,2007 French riots,Wikipedia routinely lists the ethnic make up o...,Rama,2007-11-27 17:00:46,POST,::: Does that mean that they are French ? Rama...,0.854220,12


In [5]:
def diff_filename(art):
    return "{}_diff.csv".format(art.lower().replace(' ', '_').replace('/', '_'))

In [42]:
# Calculate revert-corrected sum diffs
diff_dirpath = '../enwiki_diffs/'
sess_outcome = {}
zero_ctr = 0

# for a,t in tqdm(sorted(list(keys)[:1])):
for a,t in tqdm(sorted(keys)):
    fullpath = os.path.join(diff_dirpath, diff_filename(a))
#     print(fullpath)
    diff = pd.read_csv(fullpath, parse_dates=['timestamp'])

    sess_talk = talk[(talk['article']==a) & (talk['thread']==t)]
    sess_ts = {}
    sess_ts['beg'] = min(sess_talk['timestamp'])
    sess_ts['end'] = max(sess_talk['timestamp']) + DateOffset(days=7)
    talk_participants = set(sess_talk['editor'].unique())

    sess_diffs = diff[(diff['article_name']==a) & \
                      (diff['editor'].isin(talk_participants)) & \
                     (diff['timestamp'] > sess_ts['beg']) & (diff['timestamp'] < sess_ts['end'])]

    if len(sess_diffs)==0:
        zero_ctr += 1
        continue
        
    diff_size = sum(sess_diffs['additions'].map(lambda x: len(str(x).split()))) + \
                        sum(sess_diffs['deletions'].map(lambda x: len(str(x).split())))
        
    # Correct for reverted token changes
    reverted_toks = 0
    for i, adds, dels in zip(sess_diffs.index, sess_diffs['additions'], sess_diffs['deletions']):
        if i+1 in diff.index:
            edit_after = diff.loc[i+1]

            if isinstance(edit_after['deletions'], str) and isinstance(adds, str): # deletions added in 
                reverted_toks += len(set(adds.split()).intersection(set(edit_after['deletions'].split())))
            if isinstance(edit_after['additions'], str) and isinstance(dels, str): # additions deleted
                reverted_toks += len(set(dels.split()).intersection(set(edit_after['additions'].split())))
            
    print(reverted_toks)
    
    sess_outcome[a,t] = [diff_size - reverted_toks]

print(zero_ctr)

2
313
12
31
0
5
16
41
167
84
0
0
57
2
8
39
0
21
5
27
0
0
807
3
18
3
9
0
1722
2
370
31
21
7
2
2
22
5
31
2
6
68
9
71
37
0
0
85
0
1
0
12
4
278
81
2
0
656
37
8
62
6
665
84
34
93
3
10
2
18
50
30
1433
1
531
9
5
21
0
16
242
9
94
12
0

15


In [43]:
# Create sess_outcome dataframe, merge in
# outcomes = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/enwiki/enwiki_conversational_outcomes.csv')
outcomes = pd.read_csv('/home/michael/school/research/wp/enwiki_sample_outcomes_7days.csv')
outcomes

,article,thread,editor,timestamp,thread_starter,text,editor_score,n_article_edits,conv_size,diff_size
0,2005 New Orleans flood,Conspiracy Theories,Malleus Fatuorum,2008-05-30 12:44:13,THREAD_STARTER,While I don't buy the idea that the government...,0.490798,21,5,561
1,2005 New Orleans flood,Conspiracy Theories,Don\'tKnowItAtAll,2011-01-01 21:11:56,POST,"A bit late to reply to this subject, but Katri...",0.747573,21,5,561
2,2005 New Orleans flood,Conspiracy Theories,Sedna1000,2011-04-09 23:16:44,POST,Why is this even on Wikipedia? The flooding of...,1.000000,21,5,561
3,2005 New Orleans flood,Conspiracy Theories,Infrogmation,2011-04-10 12:36:53,POST,It's on Wikipedia because it is a rumor or fol...,0.602496,21,5,561
4,2005 New Orleans flood,Conspiracy Theories,Sedna1000,2011-04-13 01:54:18,POST,"Well maybe the article should read ""although b...",1.000000,21,5,561
5,Ann Gora,TurboKat Origins,GavinSimmons,2008-08-25 11:24:55,THREAD_STARTER,I recently added that the TurboKat most notabl...,1.000000,2,4,8
6,Ann Gora,TurboKat Origins,Lots42,2008-08-25 11:50:01,POST,I've no doubt the plane resembles such. But to...,0.808824,2,4,8
7,Ann Gora,TurboKat Origins,Renegade5115,2008-10-26 22:45:05,POST,:It also resembles Yak-141! —Preceding unsign...,1.000000,2,4,8
8,Ann Gora,TurboKat Origins,Cody-7,2008-12-07 15:57:34,POST,It most certainly does resemble an F-14. Swept...,NaN,2,4,8
9,"Apex, North Carolina",Neighborhoods,OPaul,2006-11-11 14:42:24,THREAD_STARTER,Are neighborhoods really that encyclopedic? Pe...,0.793103,11,5,215


In [44]:
# Create dataframe of conversations to merge in
outcome_colnames = ['corrected_diff_size']
sess_outcome_df = pd.DataFrame([[k[0], k[1]] + val for k,val in sess_outcome.items()], columns=['article', 'thread'] + outcome_colnames)
sess_outcome_df

,article,thread,corrected_diff_size
0,2005 New Orleans flood,Conspiracy Theories,559
1,2007 French riots,Wikipedia routinely lists the ethnic make up o...,655
2,Anaerobic digestion,Anaerobic Digesters in Hog Farms,222
3,Andrew Cotton,Is Rose leaving?,2138
4,Ann Gora,TurboKat Origins,8
5,Anti-American sentiment in Europe and Asia,European Opinion on America,48
6,"Apex, North Carolina",Neighborhoods,199
7,BC Lions,History section,11342
8,Bandidos Motorcycle Club,Support Clubs,1053
9,Beanpole family,"As for merging ""Kinship and Descent"" into ""Kin...",1424


In [45]:
merged = pd.merge(outcomes, sess_outcome_df)
merged

,article,thread,editor,timestamp,thread_starter,text,editor_score,n_article_edits,conv_size,diff_size,corrected_diff_size
0,2005 New Orleans flood,Conspiracy Theories,Malleus Fatuorum,2008-05-30 12:44:13,THREAD_STARTER,While I don't buy the idea that the government...,0.490798,21,5,561,559
1,2005 New Orleans flood,Conspiracy Theories,Don\'tKnowItAtAll,2011-01-01 21:11:56,POST,"A bit late to reply to this subject, but Katri...",0.747573,21,5,561,559
2,2005 New Orleans flood,Conspiracy Theories,Sedna1000,2011-04-09 23:16:44,POST,Why is this even on Wikipedia? The flooding of...,1.000000,21,5,561,559
3,2005 New Orleans flood,Conspiracy Theories,Infrogmation,2011-04-10 12:36:53,POST,It's on Wikipedia because it is a rumor or fol...,0.602496,21,5,561,559
4,2005 New Orleans flood,Conspiracy Theories,Sedna1000,2011-04-13 01:54:18,POST,"Well maybe the article should read ""although b...",1.000000,21,5,561,559
5,Ann Gora,TurboKat Origins,GavinSimmons,2008-08-25 11:24:55,THREAD_STARTER,I recently added that the TurboKat most notabl...,1.000000,2,4,8,8
6,Ann Gora,TurboKat Origins,Lots42,2008-08-25 11:50:01,POST,I've no doubt the plane resembles such. But to...,0.808824,2,4,8,8
7,Ann Gora,TurboKat Origins,Renegade5115,2008-10-26 22:45:05,POST,:It also resembles Yak-141! —Preceding unsign...,1.000000,2,4,8,8
8,Ann Gora,TurboKat Origins,Cody-7,2008-12-07 15:57:34,POST,It most certainly does resemble an F-14. Swept...,NaN,2,4,8,8
9,"Apex, North Carolina",Neighborhoods,OPaul,2006-11-11 14:42:24,THREAD_STARTER,Are neighborhoods really that encyclopedic? Pe...,0.793103,11,5,215,199


In [38]:
merged.columns

Index(['article', 'thread', 'editor', 'timestamp', 'thread_starter', 'text',
       'editor_score', 'n_article_edits', 'conv_size', 'diff_size',
       'corrected_diff_size'],
      dtype='object')

In [47]:
merged.to_csv('/home/michael/school/research/wp/enwiki_sample_outcomes.csv', index=False)

# See problem with Yohan's old feature extraction

In [2]:
feats = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/enwiki/conversational_outcomes_feats.csv')
feats

,article,thread,editor,ART::DEF,ART::INDEF,AUTH::CREDENTIALS,AUTH::EXPERIENTIAL,AUTH::EXTERNAL,AUTH::FORUM,AUTH::SOCIAL_EXPECTATIONS,EXAMPLE,FIRST_TURN,LAST_TURN,NUM_TURNS,PERS_PRON::PLUR,PERS_PRON::SING,QUESTION,SENTI::NEG,SENTI::POS,URL
0,"""Tiger"" William Dunlop",Current image,Canglesea,11.0,2.0,0.033549,0.004684,-0.000792,-0.000158,0.002823,0.0,1,0,1.0,0.0,7.0,0.0,0.0,10.0,0.0
1,"""Tiger"" William Dunlop",Current image,Deconstructhis,9.0,8.0,0.015125,0.006766,-0.000347,0.000688,0.002136,0.0,0,0,2.0,2.0,8.0,0.0,1.0,11.0,0.0
2,"""Tiger"" William Dunlop",Current image,Ukexpat,2.0,0.0,0.007372,0.000275,-0.000728,0.000426,-0.000247,0.0,0,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0
3,"""Tiger"" William Dunlop",Current image,Skinsmoke,0.0,0.0,0.010457,0.001906,-0.000420,0.000056,0.000092,0.0,0,0,1.0,0.0,1.0,0.0,0.0,2.0,0.0
4,"""Tiger"" William Dunlop",Requested move,Deconstructhis,3.0,4.0,0.040631,0.002054,-0.000389,0.001663,0.001624,0.0,0,0,1.0,1.0,3.0,0.0,0.0,7.0,0.0
5,"""Tiger"" William Dunlop",Requested move,Billinghurst,5.0,6.0,0.003066,0.000754,-0.000486,0.001702,0.000641,0.0,1,0,2.0,0.0,0.0,0.0,4.0,2.0,0.0
6,"""Tiger"" William Dunlop",Requested move,CJ3370,0.0,1.0,-0.000015,0.000328,-0.000107,0.000119,0.000394,0.0,0,0,2.0,0.0,1.0,0.0,0.0,1.0,0.0
7,"""Tiger"" William Dunlop",Requested move,Skinsmoke,8.0,5.0,0.004266,0.004616,0.000020,0.003166,0.001978,0.0,0,1,3.0,5.0,2.0,1.0,1.0,6.0,0.0
8,"""V"" Is for Vagina","Why is the ""i"" capitalized?",Skomorokh,18.0,3.0,0.014986,0.004507,-0.000320,0.000547,0.000835,0.0,0,1,3.0,2.0,5.0,1.0,1.0,10.0,1.0
9,"""V"" Is for Vagina","Why is the ""i"" capitalized?",Gunmetal Angel,4.0,0.0,0.002190,0.001293,-0.000320,0.000743,0.000763,0.0,1,0,1.0,0.0,4.0,0.0,1.0,3.0,0.0


In [3]:
talk = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/enwiki/enwiki_conversational_outcomes.csv')
talk

,article,thread,timestamp,editor,thread_type,text,conv_size,n_article_edits,diff_size
0,"""Tiger"" William Dunlop",Current image,2010-07-15 20:19:17,Deconstructhis,THREAD_STARTER,I've noticed that the current image that's acc...,5,14,253
1,"""Tiger"" William Dunlop",Current image,2010-07-15 20:30:15,Ukexpat,POST,I removed the deletion tag because the image i...,5,14,253
2,"""Tiger"" William Dunlop",Current image,2010-07-15 20:34:20,Deconstructhis,POST,:Excellent! Thank you very much for your help ...,5,14,253
3,"""Tiger"" William Dunlop",Current image,2010-07-16 23:41:08,Skinsmoke,POST,::Good move ukexpat. That was really helpful ...,5,14,253
4,"""Tiger"" William Dunlop",Current image,2011-04-02 11:35:48,Canglesea,POST,"Hello, I am not a great user of the computer b...",5,14,253
5,"""Tiger"" William Dunlop",Requested move,2010-07-14 10:47:31,CJ3370,POST,"William ""Tiger"" Dunlop → William ""Tiger"" Dunlo...",8,44,2605
6,"""Tiger"" William Dunlop",Requested move,2010-07-14 10:47:31,CJ3370,POST,I'd like to make this contribution to Tiger Du...,8,44,2605
7,"""Tiger"" William Dunlop",Requested move,2010-07-14 15:12:03,Skinsmoke,POST,Oppose This will result in two articles on th...,8,44,2605
8,"""Tiger"" William Dunlop",Requested move,2010-07-14 16:59:04,Deconstructhis,POST,Not a problem to merge if that is the most app...,8,44,2605
9,"""Tiger"" William Dunlop",Requested move,2010-07-14 19:49:58,Skinsmoke,POST,User:CJ3370/William_Tiger_Dunlop → William Tig...,8,44,2605


# Examine conversational outcome data

In [7]:
data = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/enwiki/enwiki_conversational_outcomes.csv', parse_dates=['timestamp'])
data

,article,thread,timestamp,editor,thread_type,text,conv_size,n_article_edits,diff_size
0,"""Tiger"" William Dunlop",Current image,2011-04-02 11:35:48,Canglesea,POST,"Hello, I am not a great user of the computer b...",5,14,253
1,"""Tiger"" William Dunlop",Current image,2010-07-15 20:19:17,Deconstructhis,THREAD_STARTER,I've noticed that the current image that's acc...,5,14,253
2,"""Tiger"" William Dunlop",Current image,2010-07-15 20:34:20,Deconstructhis,POST,:Excellent! Thank you very much for your help ...,5,14,253
3,"""Tiger"" William Dunlop",Current image,2010-07-16 23:41:08,Skinsmoke,POST,::Good move ukexpat. That was really helpful ...,5,14,253
4,"""Tiger"" William Dunlop",Current image,2010-07-15 20:30:15,Ukexpat,POST,I removed the deletion tag because the image i...,5,14,253
5,"""Tiger"" William Dunlop",Requested move,2010-07-21 17:13:40,Billinghurst,THREAD_STARTER,The following discussion is an archived discus...,8,44,2605
6,"""Tiger"" William Dunlop",Requested move,2010-07-21 17:13:40,Billinghurst,POST,The above discussion is preserved as an archiv...,8,44,2605
7,"""Tiger"" William Dunlop",Requested move,2010-07-14 10:47:31,CJ3370,POST,"William ""Tiger"" Dunlop → William ""Tiger"" Dunlo...",8,44,2605
8,"""Tiger"" William Dunlop",Requested move,2010-07-14 10:47:31,CJ3370,POST,I'd like to make this contribution to Tiger Du...,8,44,2605
9,"""Tiger"" William Dunlop",Requested move,2010-07-14 16:59:04,Deconstructhis,POST,Not a problem to merge if that is the most app...,8,44,2605


In [3]:
print(len(data))
print(len(set(zip(data['article'], data['thread']))))

179925
41662


In [4]:
for col in ['conv_size', 'n_article_edits', 'diff_size']:
    print(data[col].count())

179925
179925
179925


In [8]:
data.sort_values(['article', 'thread', 'timestamp'], inplace=True)

In [9]:
data.to_csv('/home/michael/school/research/wp/wikipedia/data/enwiki/enwiki_conversational_outcomes.csv', index=False)

# Calculate outcome measure: number of edits and diff sizes during and after a discussion

In [6]:
# samp = pd.read_csv('/home/michael/school/research/wp/enwiki_qualitative_analysis.csv', parse_dates=['timestamp'])
# talk = samp
talk = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/all_talk.csv', 
                   names=['id', 'article', 'thread', 'timestamp', 'editor', 'thread_type', 'text'],
                  parse_dates=['timestamp'])
talk.drop(['id'], axis=1, inplace=True)

keys = set(zip(talk['article'], talk['thread']))
talk

,article,thread,timestamp,editor,thread_type,text
0,Acharya S,POV: undue bias towards criticism,2010-11-21 06:44:51,^^James^^,THREAD_STARTER,http://en.wikipedia.org/wiki/Kenneth_Feder\nPr...
1,Acharya S,POV: undue bias towards criticism,2010-11-21 15:36:50,^^James^^,POST,::You mean the assertion that Price is support...
2,Da Drought 2,Children,2009-03-10 18:01:07,^-stephaanie,THREAD_STARTER,"Yeah man, Dwayne only has one daughter and one..."
3,Lil Tunechi,Children,2009-03-10 18:01:07,^-stephaanie,THREAD_STARTER,"Yeah man, Dwayne only has one daughter and one..."
4,Americans with Disabilities Act of 1990,Neutrality?,2005-03-10 16:06:30,^demon,POST,"In my personal opinion, this article is not ve..."
5,Americans with Disabilities Act of 1990/Defini...,Neutrality?,2005-03-10 16:06:30,^demon,POST,"In my personal opinion, this article is not ve..."
6,Google bomb,"Google search ""talentless hack"" rating",2006-12-20 02:23:06,^demon,THREAD_STARTER,The page claims that it itself is currently th...
7,Montréal–Pierre Elliott Trudeau International ...,Controvercy,2007-03-25 11:09:38,^demonBot2,THREAD_STARTER,I work at YUL and there is and has been free w...
8,Thule Air Base,troops,2007-03-25 11:07:46,^demonBot2,THREAD_STARTER,how many troops are stationed there
9,Mobile PCI Express Module,Failure,2007-06-22 05:39:10,- - -,THREAD_STARTER,Nvidia just sell the chips for the MXM modules...


In [8]:
# Filter all_talk
conv_sizes = talk.groupby(['article', 'thread']).size()
conv_sizes.name = 'conv_size'
talk = talk.join(conv_sizes, on=['article', 'thread'])
talk

,article,thread,timestamp,editor,thread_type,text,conv_size
0,Acharya S,POV: undue bias towards criticism,2010-11-21 06:44:51,^^James^^,THREAD_STARTER,http://en.wikipedia.org/wiki/Kenneth_Feder\nPr...,4
1,Acharya S,POV: undue bias towards criticism,2010-11-21 15:36:50,^^James^^,POST,::You mean the assertion that Price is support...,4
2,Da Drought 2,Children,2009-03-10 18:01:07,^-stephaanie,THREAD_STARTER,"Yeah man, Dwayne only has one daughter and one...",3
3,Lil Tunechi,Children,2009-03-10 18:01:07,^-stephaanie,THREAD_STARTER,"Yeah man, Dwayne only has one daughter and one...",3
4,Americans with Disabilities Act of 1990,Neutrality?,2005-03-10 16:06:30,^demon,POST,"In my personal opinion, this article is not ve...",8
5,Americans with Disabilities Act of 1990/Defini...,Neutrality?,2005-03-10 16:06:30,^demon,POST,"In my personal opinion, this article is not ve...",8
6,Google bomb,"Google search ""talentless hack"" rating",2006-12-20 02:23:06,^demon,THREAD_STARTER,The page claims that it itself is currently th...,2
7,Montréal–Pierre Elliott Trudeau International ...,Controvercy,2007-03-25 11:09:38,^demonBot2,THREAD_STARTER,I work at YUL and there is and has been free w...,1
8,Thule Air Base,troops,2007-03-25 11:07:46,^demonBot2,THREAD_STARTER,how many troops are stationed there,2
9,Mobile PCI Express Module,Failure,2007-06-22 05:39:10,- - -,THREAD_STARTER,Nvidia just sell the chips for the MXM modules...,1


In [9]:
talk.sort_values(['article', 'thread'], inplace=True)
talk

,article,thread,timestamp,editor,thread_type,text,conv_size
37009,"""Tiger"" William Dunlop",Current image,2011-04-02 11:35:48,Canglesea,POST,"Hello, I am not a great user of the computer b...",5
54194,"""Tiger"" William Dunlop",Current image,2010-07-15 20:19:17,Deconstructhis,THREAD_STARTER,I've noticed that the current image that's acc...,5
54195,"""Tiger"" William Dunlop",Current image,2010-07-15 20:34:20,Deconstructhis,POST,:Excellent! Thank you very much for your help ...,5
190540,"""Tiger"" William Dunlop",Current image,2010-07-16 23:41:08,Skinsmoke,POST,::Good move ukexpat. That was really helpful ...,5
215603,"""Tiger"" William Dunlop",Current image,2010-07-15 20:30:15,Ukexpat,POST,I removed the deletion tag because the image i...,5
28483,"""Tiger"" William Dunlop",Requested move,2010-07-21 17:13:40,Billinghurst,THREAD_STARTER,The following discussion is an archived discus...,8
28484,"""Tiger"" William Dunlop",Requested move,2010-07-21 17:13:40,Billinghurst,POST,The above discussion is preserved as an archiv...,8
43047,"""Tiger"" William Dunlop",Requested move,2010-07-14 10:47:31,CJ3370,POST,"William ""Tiger"" Dunlop → William ""Tiger"" Dunlo...",8
43048,"""Tiger"" William Dunlop",Requested move,2010-07-14 10:47:31,CJ3370,POST,I'd like to make this contribution to Tiger Du...,8
54193,"""Tiger"" William Dunlop",Requested move,2010-07-14 16:59:04,Deconstructhis,POST,Not a problem to merge if that is the most app...,8


In [10]:
print(len(talk))
talk = talk[talk['conv_size']>=2]
print(len(talk))

237207
183690


In [11]:
talk.to_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/talk_conversational_outcomes.csv', index=False)

In [12]:
def diff_filename(art):
    return "{}_diff.csv".format(art.lower().replace(' ', '_').replace('/', '_'))

In [51]:
# Calculate n_edits
diff_dirpath = '../enwiki_diffs/'
# sess_outcome = {'n_article_edits': {}, 'diff_size': {}}
sess_outcome = {}
zero_ctr = 0

for a,t in tqdm(sorted(keys)):
    fullpath = os.path.join(diff_dirpath, diff_filename(a))
#     print(fullpath)
    diff = pd.read_csv(fullpath, parse_dates=['timestamp'])

    sess_talk = talk[(talk['article']==a) & (talk['thread']==t)]
    sess_ts = {}
    sess_ts['beg'] = min(sess_talk['timestamp'])
    sess_ts['end'] = max(sess_talk['timestamp']) + DateOffset(days=7)
    talk_participants = set(sess_talk['editor'].unique())

    sess_diffs = diff[(diff['article_name']==a) & \
                      (diff['editor'].isin(talk_participants)) & \
                     (diff['timestamp'] > sess_ts['beg']) & (diff['timestamp'] < sess_ts['end'])]
    if len(sess_diffs)==0:
        zero_ctr += 1

    n_article_edits = len(sess_diffs)
    diff_size = sum(sess_diffs['additions'].map(lambda x: len(str(x).split()))) + \
                        sum(sess_diffs['deletions'].map(lambda x: len(str(x).split())))
    
    sess_outcome[a,t] = [n_article_edits, diff_size]

print(zero_ctr)


15


In [56]:
# Create sess_outcome dataframe, merge in
outcome_colnames = ['n_article_edits', 'diff_size']
sess_outcome_df = pd.DataFrame([[k[0], k[1]] + val for k,val in sess_outcome.items()], columns=['article', 'thread'] + outcome_colnames)
sess_outcome_df

,article,thread,n_article_edits,diff_size
0,2005 New Orleans flood,Conspiracy Theories,21,561
1,2007 French riots,Wikipedia routinely lists the ethnic make up o...,24,968
2,Anaerobic digestion,Anaerobic Digesters in Hog Farms,31,234
3,Andrew Cotton,Is Rose leaving?,19,2169
4,Ann Gora,TurboKat Origins,2,8
5,Anti-American sentiment in Europe and Asia,European Opinion on America,1,53
6,"Apex, North Carolina",Neighborhoods,11,215
7,BC Lions,History section,85,11383
8,Bandidos Motorcycle Club,Support Clubs,59,1220
9,Beanpole family,"As for merging ""Kinship and Descent"" into ""Kin...",49,1508


In [57]:
# merged = pd.merge(samp, sess_outcome_df)
merged = pd.merge(new_df, sess_outcome_df)
merged

,article,thread,editor,timestamp,thread_starter,text,editor_score,n_article_edits,conv_size,diff_size
0,2005 New Orleans flood,Conspiracy Theories,Malleus Fatuorum,2008-05-30 12:44:13,THREAD_STARTER,While I don't buy the idea that the government...,0.490798,21,5,561
1,2005 New Orleans flood,Conspiracy Theories,Don\'tKnowItAtAll,2011-01-01 21:11:56,POST,"A bit late to reply to this subject, but Katri...",0.747573,21,5,561
2,2005 New Orleans flood,Conspiracy Theories,Sedna1000,2011-04-09 23:16:44,POST,Why is this even on Wikipedia? The flooding of...,1.000000,21,5,561
3,2005 New Orleans flood,Conspiracy Theories,Infrogmation,2011-04-10 12:36:53,POST,It's on Wikipedia because it is a rumor or fol...,0.602496,21,5,561
4,2005 New Orleans flood,Conspiracy Theories,Sedna1000,2011-04-13 01:54:18,POST,"Well maybe the article should read ""although b...",1.000000,21,5,561
5,Ann Gora,TurboKat Origins,GavinSimmons,2008-08-25 11:24:55,THREAD_STARTER,I recently added that the TurboKat most notabl...,1.000000,2,4,8
6,Ann Gora,TurboKat Origins,Lots42,2008-08-25 11:50:01,POST,I've no doubt the plane resembles such. But to...,0.808824,2,4,8
7,Ann Gora,TurboKat Origins,Renegade5115,2008-10-26 22:45:05,POST,:It also resembles Yak-141! —Preceding unsign...,1.000000,2,4,8
8,Ann Gora,TurboKat Origins,Cody-7,2008-12-07 15:57:34,POST,It most certainly does resemble an F-14. Swept...,NaN,2,4,8
9,"Apex, North Carolina",Neighborhoods,OPaul,2006-11-11 14:42:24,THREAD_STARTER,Are neighborhoods really that encyclopedic? Pe...,0.793103,11,5,215


In [23]:
conv_sizes

article                                     thread                                                       
2005 New Orleans flood                      Conspiracy Theories                                               5
2007 French riots                           Wikipedia routinely lists the ethnic make up of individuals      12
Anaerobic digestion                         Anaerobic Digesters in Hog Farms                                  2
Andrew Cotton                               Is Rose leaving?                                                  9
Ann Gora                                    TurboKat Origins                                                  4
Anti-American sentiment in Europe and Asia  European Opinion on America                                       2
Apex, North Carolina                        Neighborhoods                                                     5
BC Lions                                    History section                                                   

In [24]:
merged.columns

Index(['article', 'thread', 'editor', 'timestamp', 'thread_starter', 'text',
       'editor_score', 'n_article_edits'],
      dtype='object')

In [58]:
# new_df.to_csv('../enwiki_sample_outcomes.csv', index=False)
merged.to_csv('../enwiki_sample_outcomes_7days.csv', index=False)

In [5]:
# Write out sample article names
samp_arts = sorted(samp['article'].unique().tolist())
print(len(samp_arts))

with open('../enwiki_sample_articles.txt', 'w') as out:
    for art in samp_arts:
        out.write("{}_diff.csv\n".format(art.lower().replace(' ', '_').replace('/', '_')))

99

In [37]:
merged

,article,thread,editor,timestamp,thread_starter,text,editor_score,n_article_edits,conv_size,diff_size
0,2005 New Orleans flood,Conspiracy Theories,Malleus Fatuorum,2008-05-30 12:44:13,THREAD_STARTER,While I don't buy the idea that the government...,0.490798,21,5,561
1,2005 New Orleans flood,Conspiracy Theories,Don\'tKnowItAtAll,2011-01-01 21:11:56,POST,"A bit late to reply to this subject, but Katri...",0.747573,21,5,561
2,2005 New Orleans flood,Conspiracy Theories,Sedna1000,2011-04-09 23:16:44,POST,Why is this even on Wikipedia? The flooding of...,1.000000,21,5,561
3,2005 New Orleans flood,Conspiracy Theories,Infrogmation,2011-04-10 12:36:53,POST,It's on Wikipedia because it is a rumor or fol...,0.602496,21,5,561
4,2005 New Orleans flood,Conspiracy Theories,Sedna1000,2011-04-13 01:54:18,POST,"Well maybe the article should read ""although b...",1.000000,21,5,561
5,2007 French riots,Wikipedia routinely lists the ethnic make up o...,Mnw2000,2007-11-27 15:57:40,THREAD_STARTER,Take any simple article about an individual an...,NaN,13,12,180
6,2007 French riots,Wikipedia routinely lists the ethnic make up o...,Rama,2007-11-27 15:59:35,POST,"I you are referring to the victims, as for now...",0.854220,13,12,180
7,2007 French riots,Wikipedia routinely lists the ethnic make up o...,Meithal,2007-11-27 16:03:45,POST,: Rame : see http://jt.france2.fr/20h/ Meithal...,NaN,13,12,180
8,2007 French riots,Wikipedia routinely lists the ethnic make up o...,RenniePet,2007-11-27 16:59:12,POST,"::The BBC article says, at the very bottom, th...",0.857143,13,12,180
9,2007 French riots,Wikipedia routinely lists the ethnic make up o...,Rama,2007-11-27 17:00:46,POST,::: Does that mean that they are French ? Rama...,0.854220,13,12,180


## Correlations

In [38]:
pearsonr(merged['conv_size'], merged['n_article_edits'])

(0.4000961127375065, 3.5149676958626493e-21)

In [39]:
pearsonr(merged['n_article_edits'], merged['diff_size'])

(0.36529166116945744, 1.1372008983021534e-17)

## Face validity check

In [40]:
# Face validity for n_edits
n_edits_sorted = merged.sort_values(['n_article_edits'], ascending=False)
n_edits_sorted

,article,thread,editor,timestamp,thread_starter,text,editor_score,n_article_edits,conv_size,diff_size
310,List of songs by Mariah Carey,We cannot give her age without citation,Tenebrae,2013-07-11 09:05:00,POST,": As well, her own Long Island newspaper, News...",0.335968,106,25,1599
303,List of songs by Mariah Carey,We cannot give her age without citation,Tenebrae,2011-01-31 10:13:38,POST,": Without citation, someone wrote in an edit s...",0.335968,106,25,1599
314,List of songs by Mariah Carey,We cannot give her age without citation,Teammm,2013-07-11 20:50:21,POST,Mariah Carey early career interview in 1990 - ...,0.618812,106,25,1599
313,List of songs by Mariah Carey,We cannot give her age without citation,Tenebrae,2013-07-11 16:38:05,POST,Fine. I'm not going to argue or revert. Howeve...,0.335968,106,25,1599
312,List of songs by Mariah Carey,We cannot give her age without citation,Teammm,2013-07-11 14:42:57,POST,2013 A+E Networks Biography - confirms full bi...,0.618812,106,25,1599
311,List of songs by Mariah Carey,We cannot give her age without citation,Teammm,2013-07-11 14:14:20,POST,All of these sources which are current and spe...,0.618812,106,25,1599
322,List of songs by Mariah Carey,We cannot give her age without citation,Tenebrae,2014-05-22 03:58:50,POST,:: Please see here- http://photos.toofab.com/g...,0.335968,106,25,1599
309,List of songs by Mariah Carey,We cannot give her age without citation,Tenebrae,2013-07-11 08:45:36,POST,:: That page doesn't show any documents.,0.335968,106,25,1599
308,List of songs by Mariah Carey,We cannot give her age without citation,Jjmcspooh,2011-05-19 02:35:22,POST,The twins' birth certificates have just been r...,0.500000,106,25,1599
307,List of songs by Mariah Carey,We cannot give her age without citation,PoppleGumBop,2011-05-17 04:15:30,POST,[ ] Here is an interview promoting debut albu...,NaN,106,25,1599


In [45]:
# Look into 0s
zeros = merged[merged['n_article_edits']==0]
print(len(set(zip(zeros['article'], zeros['thread']))))
zeros

18


,article,thread,editor,timestamp,thread_starter,text,editor_score,n_article_edits,conv_size,diff_size
32,Anti-American sentiment in Europe and Asia,European Opinion on America,Rjensen,2014-02-02 17:12:27,THREAD_STARTER,"According to latest polls in Slovenia (2013), ...",0.604174,0,2,0
33,Anti-American sentiment in Europe and Asia,European Opinion on America,The Four Deuces,2014-02-02 20:06:26,POST,Opposition to U.S. foreign policy is not neces...,0.332604,0,2,0
67,Chandra Nalaar,Skills required: Common Sense,Ian.thomson,2010-05-27 19:47:40,THREAD_STARTER,This is a little strange for an infobox's cont...,NaN,0,3,0
68,Chandra Nalaar,Skills required: Common Sense,OdinFK,2010-06-01 16:09:25,POST,:I play the game a lot and while I don't reall...,0.249628,0,3,0
69,Chandra Nalaar,Skills required: Common Sense,Croctotheface,2010-07-18 19:26:25,POST,"Since someone readded this, I'll just add my v...",0.078300,0,3,0
86,Daultay Dofine,In all due seriousness...,New Age Retro Hippie,2011-02-21 19:10:16,THREAD_STARTER,This needs to be trimmed massively. We need to...,0.000000,0,21,0
87,Daultay Dofine,In all due seriousness...,Harry Blue5,2011-02-22 15:43:13,POST,I think we should make it so that characters h...,0.541551,0,21,0
88,Daultay Dofine,In all due seriousness...,New Age Retro Hippie,2011-03-07 16:42:21,POST,":We can't use a ""number of appearances"" guidel...",0.000000,0,21,0
89,Daultay Dofine,In all due seriousness...,Harry Blue5,2011-03-07 17:02:56,POST,::This would be so much easier if the EU hadn'...,0.541551,0,21,0
90,Daultay Dofine,In all due seriousness...,Blake,2011-03-07 17:09:25,POST,"Well, it all depends on multiple factors. Woul...",0.846154,0,21,0


In [44]:
for a,t in sorted(set(zip(zeros['article'], zeros['thread']))):
    print(a)
    print(t)
    fullpath = os.path.join(diff_dirpath, diff_filename(a))
    diff = pd.read_csv(fullpath, parse_dates=['timestamp'])

    sess_talk = talk[(talk['article']==a) & (talk['thread']==t)]
    sess_ts = {}
    sess_ts['beg'] = min(sess_talk['timestamp'])
    sess_ts['end'] = max(sess_talk['timestamp']) + DateOffset(days=1)
    talk_participants = set(sess_talk['editor'].unique())
    print(sess_ts)

    sess_diffs = diff[(diff['article_name']==a) & \
                      (diff['editor'].isin(talk_participants)) & \
                     (diff['timestamp'] > sess_ts['beg']) & (diff['timestamp'] < sess_ts['end'])]
    print(len(sess_diffs))

    sess_outcome[a,t] = sum(sess_diffs['additions'].map(lambda x: len(str(x).split()))) + \
                        sum(sess_diffs['deletions'].map(lambda x: len(str(x).split())))
    print()

Anti-American sentiment in Europe and Asia
European Opinion on America
{'beg': Timestamp('2014-02-02 17:12:27'), 'end': Timestamp('2014-02-03 20:06:26')}
0

Chandra Nalaar
Skills required: Common Sense
{'beg': Timestamp('2010-05-27 19:47:40'), 'end': Timestamp('2010-07-19 19:26:25')}
0

Daultay Dofine
In all due seriousness...
{'beg': Timestamp('2011-02-21 19:10:16'), 'end': Timestamp('2011-03-10 15:55:27')}
0

Falaise pocket
Operation Totalize
{'beg': Timestamp('2009-03-15 11:08:56'), 'end': Timestamp('2009-03-17 12:13:07')}
0

Flag of Sealand
WP:N
{'beg': Timestamp('2010-12-27 19:47:57'), 'end': Timestamp('2010-12-28 21:15:17')}
0

Fursona
What does the fox say
{'beg': Timestamp('2013-10-28 21:02:58'), 'end': Timestamp('2013-10-30 11:35:03')}
0

Hair
Baldwin/Brando photo
{'beg': Timestamp('2006-04-09 23:28:44'), 'end': Timestamp('2006-04-13 20:53:46')}
0

Hammer Brothers' Suit
Marsha Kinder content
{'beg': Timestamp('2009-05-30 00:31:24'), 'end': Timestamp('2009-05-31 08:48:02')}
0



# Try to predict editor score based on talk by myself--see what's going on in conversations

In [3]:
talk = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/all_talk.csv', header=None, names=['id','article', 'thread', 'timestamp', 'editor', 'thread_starter', 'text'])
print(len(talk))

data = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_scores_roles.csv')

merged = pd.merge(talk, data, how='left')
print(len(merged))
merged.columns

237207
237207


Index(['id', 'article', 'thread', 'timestamp', 'editor', 'thread_starter',
       'text', 'split', 'fold', 'editor_talk', 'other_talk', '#editor_turns',
       '#other_turns', 'ftopic0', 'ftopic1', 'ftopic2', 'ftopic3', 'ftopic4',
       'ftopic5', 'ftopic6', 'ftopic7', 'ftopic8', 'ftopic9', 'ftopic10',
       'ftopic11', 'ftopic12', 'ftopic13', 'ftopic14', 'ftopic15', 'ftopic16',
       'ftopic17', 'ftopic18', 'ftopic19', 'other_ftopic0', 'other_ftopic1',
       'other_ftopic2', 'other_ftopic3', 'other_ftopic4', 'other_ftopic5',
       'other_ftopic6', 'other_ftopic7', 'other_ftopic8', 'other_ftopic9',
       'other_ftopic10', 'other_ftopic11', 'other_ftopic12', 'other_ftopic13',
       'other_ftopic14', 'other_ftopic15', 'other_ftopic16', 'other_ftopic17',
       'other_ftopic18', 'other_ftopic19', 'ART::DEF', 'ART::INDEF',
       'AUTH::CREDENTIALS', 'AUTH::EXPERIENTIAL', 'AUTH::EXTERNAL',
       'AUTH::FORUM', 'AUTH::SOCIAL_EXPECTATIONS', 'EXAMPLE', 'FIRST_TURN',
       'LAST_TURN'

In [4]:
merged = merged.loc[:, ['article', 'thread', 'editor', 'timestamp', 'thread_starter', 'text', 'editor_score']]
scored = merged[np.isfinite(merged['editor_score'])]

threads = set(zip(scored['article'], scored['thread']))
s = random.sample(threads, 100)

mask = list(map(lambda tup: tup in s, zip(merged['article'], merged['thread'])))
sort = merged[mask].sort_values(['article', 'thread', 'timestamp'], inplace=False)
sort

,article,thread,editor,timestamp,thread_starter,text,editor_score
132099,2005 New Orleans flood,Conspiracy Theories,Malleus Fatuorum,2008-05-30 12:44:13,THREAD_STARTER,While I don't buy the idea that the government...,0.490798
59215,2005 New Orleans flood,Conspiracy Theories,Don\'tKnowItAtAll,2011-01-01 21:11:56,POST,"A bit late to reply to this subject, but Katri...",0.747573
185740,2005 New Orleans flood,Conspiracy Theories,Sedna1000,2011-04-09 23:16:44,POST,Why is this even on Wikipedia? The flooding of...,1.000000
97468,2005 New Orleans flood,Conspiracy Theories,Infrogmation,2011-04-10 12:36:53,POST,It's on Wikipedia because it is a rumor or fol...,0.602496
185741,2005 New Orleans flood,Conspiracy Theories,Sedna1000,2011-04-13 01:54:18,POST,"Well maybe the article should read ""although b...",1.000000
143393,2007 French riots,Wikipedia routinely lists the ethnic make up o...,Mnw2000,2007-11-27 15:57:40,THREAD_STARTER,Take any simple article about an individual an...,NaN
171091,2007 French riots,Wikipedia routinely lists the ethnic make up o...,Rama,2007-11-27 15:59:35,POST,"I you are referring to the victims, as for now...",0.854220
138042,2007 French riots,Wikipedia routinely lists the ethnic make up o...,Meithal,2007-11-27 16:03:45,POST,: Rame : see http://jt.france2.fr/20h/ Meithal...,NaN
173664,2007 French riots,Wikipedia routinely lists the ethnic make up o...,RenniePet,2007-11-27 16:59:12,POST,"::The BBC article says, at the very bottom, th...",0.857143
171092,2007 French riots,Wikipedia routinely lists the ethnic make up o...,Rama,2007-11-27 17:00:46,POST,::: Does that mean that they are French ? Rama...,0.854220


In [5]:
sort.to_csv('/home/michael/school/research/wp/enwiki_qualitative_analysis.csv', index=False)

In [30]:
pd.set_option('display.max_colwidth', 999)

# Check into #editors discrepancy

In [ ]:
data = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_scores_roles.csv')

In [3]:
data[(data['article']=='Western Sahara conflict') & (data['thread']=='Merge')]

,split,fold,article,thread,editor,#editors,editor_talk,other_talk,#editor_turns,#other_turns,...,TargetRole_1,TargetRole_2,TargetRole_3,TargetRole_4,NontargetRole_0,NontargetRole_1,NontargetRole_2,NontargetRole_3,NontargetRole_4,editor_score
16827,test,0,Western Sahara conflict,Merge,Charles Essie,4,This article could merged with the Western Sah...,Oppose: I hope people sometime could understan...,1,5,...,0.138350,0.211057,0.130170,0.400538,0.504108,0.177680,0.095829,0.117891,0.104492,0.947368
16828,test,0,Western Sahara conflict,Merge,HCPUNXKID,4,Oppose: I hope people sometime could understan...,This article could merged with the Western Sah...,2,4,...,0.158112,0.078939,0.048652,0.024016,0.218126,0.143151,0.175916,0.155970,0.306837,0.441860
16829,test,0,Western Sahara conflict,Merge,Omar-toons,4,Support: Both articles have sth like 90% of th...,This article could merged with the Western Sah...,1,5,...,0.187932,0.087814,0.172614,0.135013,0.403979,0.172620,0.150362,0.067892,0.205147,0.061120


In [4]:
data[(data['article']=='Western Sahara conflict')]

,split,fold,article,thread,editor,#editors,editor_talk,other_talk,#editor_turns,#other_turns,...,TargetRole_1,TargetRole_2,TargetRole_3,TargetRole_4,NontargetRole_0,NontargetRole_1,NontargetRole_2,NontargetRole_3,NontargetRole_4,editor_score
4708,test,0,Western Sahara conflict,Algeria is not a part of the conflict,Dzlinker,2,There is no military implication of Algeria. I...,"directly involved in 1976. Regards, (talk) 01:...",1,1,...,0.054840,0.150979,0.043720,0.681921,0.093351,0.096581,0.563199,0.065765,0.181103,0.400000
4709,test,0,Western Sahara conflict,Algeria is not a part of the conflict,Omar-toons,2,"directly involved in 1976. Regards, (talk) 01:...",There is no military implication of Algeria. I...,1,1,...,0.141087,0.494757,0.087193,0.154595,0.053917,0.058734,0.153841,0.040954,0.692553,0.088063
16827,test,0,Western Sahara conflict,Merge,Charles Essie,4,This article could merged with the Western Sah...,Oppose: I hope people sometime could understan...,1,5,...,0.138350,0.211057,0.130170,0.400538,0.504108,0.177680,0.095829,0.117891,0.104492,0.947368
16828,test,0,Western Sahara conflict,Merge,HCPUNXKID,4,Oppose: I hope people sometime could understan...,This article could merged with the Western Sah...,2,4,...,0.158112,0.078939,0.048652,0.024016,0.218126,0.143151,0.175916,0.155970,0.306837,0.441860
16829,test,0,Western Sahara conflict,Merge,Omar-toons,4,Support: Both articles have sth like 90% of th...,This article could merged with the Western Sah...,1,5,...,0.187932,0.087814,0.172614,0.135013,0.403979,0.172620,0.150362,0.067892,0.205147,0.061120
43119,test,0,Western Sahara conflict,Rename,Greyshark09,2,I propose to rename the article Western Sahara...,The following discussion is an archived discus...,3,4,...,0.042757,0.646127,0.078822,0.132690,0.128352,0.109159,0.633865,0.072580,0.056045,0.836499
43120,test,0,Western Sahara conflict,Rename,Omar-toons,2,Oppose : Western Sahara conflict was a #REDIRE...,The following discussion is an archived discus...,2,5,...,0.107224,0.712667,0.046181,0.055876,0.105954,0.051776,0.570737,0.138663,0.132869,0.058376
46727,test,0,Western Sahara conflict,POV template added...,Koavf,4,Propaganda Talking about police brutality and ...,... because of the adding of this photo montag...,2,4,...,0.107874,0.708133,0.050081,0.050404,0.274151,0.166862,0.396511,0.054909,0.107566,0.823529
46728,test,0,Western Sahara conflict,POV template added...,Omar-toons,4,... because of the adding of this photo montag...,Propaganda Talking about police brutality and ...,2,4,...,0.113351,0.453108,0.086487,0.189404,0.245541,0.157367,0.487888,0.047468,0.061737,0.063116
46729,test,0,Western Sahara conflict,POV template added...,Zaldax,4,"Template removed, along with image. The entire...",... because of the adding of this photo montag...,1,5,...,0.245784,0.221329,0.047435,0.052731,0.108214,0.107485,0.585961,0.067335,0.131005,0.815822


## Recalculate #editors cols

In [6]:
gped = data.groupby(['article', 'thread'])
gped.size()[('Western Sahara conflict', 'Merge')]

3

In [7]:
convo_size = pd.DataFrame([(row[0][0], row[0][1], row[1]) for row in gped.size().iteritems()], columns=['article', 'thread', '#editors'])
len(convo_size)

21108

In [9]:
data.drop('#editors', axis=1, inplace=True)

In [10]:
merged = pd.merge(data, convo_size)
print(len(merged))
merged.columns

53175


Index(['split', 'fold', 'article', 'thread', 'editor', 'editor_talk',
       'other_talk', '#editor_turns', '#other_turns', 'ftopic0', 'ftopic1',
       'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6', 'ftopic7',
       'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12', 'ftopic13',
       'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18', 'ftopic19',
       'other_ftopic0', 'other_ftopic1', 'other_ftopic2', 'other_ftopic3',
       'other_ftopic4', 'other_ftopic5', 'other_ftopic6', 'other_ftopic7',
       'other_ftopic8', 'other_ftopic9', 'other_ftopic10', 'other_ftopic11',
       'other_ftopic12', 'other_ftopic13', 'other_ftopic14', 'other_ftopic15',
       'other_ftopic16', 'other_ftopic17', 'other_ftopic18', 'other_ftopic19',
       'ART::DEF', 'ART::INDEF', 'AUTH::CREDENTIALS', 'AUTH::EXPERIENTIAL',
       'AUTH::EXTERNAL', 'AUTH::FORUM', 'AUTH::SOCIAL_EXPECTATIONS', 'EXAMPLE',
       'FIRST_TURN', 'LAST_TURN', 'NUM_TURNS', 'PERS_PRON::PLUR',
       'PERS_PRON:

In [11]:
merged.to_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_scores_roles.csv', index=False)

In [12]:
feats = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_allfeats.csv')
feats.drop('#editors', axis=1, inplace=True)
len(feats)

53175

In [13]:
merged = pd.merge(feats, convo_size)
print(len(merged))
merged.columns

53175


Index(['split', 'fold', 'article', 'thread', 'editor', 'editor_talk',
       'other_talk', '#editor_turns', '#other_turns', 'ftopic0', 'ftopic1',
       'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6', 'ftopic7',
       'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12', 'ftopic13',
       'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18', 'ftopic19',
       'other_ftopic0', 'other_ftopic1', 'other_ftopic2', 'other_ftopic3',
       'other_ftopic4', 'other_ftopic5', 'other_ftopic6', 'other_ftopic7',
       'other_ftopic8', 'other_ftopic9', 'other_ftopic10', 'other_ftopic11',
       'other_ftopic12', 'other_ftopic13', 'other_ftopic14', 'other_ftopic15',
       'other_ftopic16', 'other_ftopic17', 'other_ftopic18', 'other_ftopic19',
       'ART::DEF', 'ART::INDEF', 'AUTH::CREDENTIALS', 'AUTH::EXPERIENTIAL',
       'AUTH::EXTERNAL', 'AUTH::FORUM', 'AUTH::SOCIAL_EXPECTATIONS', 'EXAMPLE',
       'FIRST_TURN', 'LAST_TURN', 'NUM_TURNS', 'PERS_PRON::PLUR',
       'PERS_PRON:

In [14]:
merged.to_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_allfeats.csv', index=False)

# Examine role distributions in conversations

In [2]:
data = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_scores_roles.csv')
print(len(data))
data.columns

53175


Index(['split', 'fold', 'article', 'thread', 'editor', '#editors',
       'editor_talk', 'other_talk', '#editor_turns', '#other_turns', 'ftopic0',
       'ftopic1', 'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6',
       'ftopic7', 'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12',
       'ftopic13', 'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18',
       'ftopic19', 'other_ftopic0', 'other_ftopic1', 'other_ftopic2',
       'other_ftopic3', 'other_ftopic4', 'other_ftopic5', 'other_ftopic6',
       'other_ftopic7', 'other_ftopic8', 'other_ftopic9', 'other_ftopic10',
       'other_ftopic11', 'other_ftopic12', 'other_ftopic13', 'other_ftopic14',
       'other_ftopic15', 'other_ftopic16', 'other_ftopic17', 'other_ftopic18',
       'other_ftopic19', 'ART::DEF', 'ART::INDEF', 'AUTH::CREDENTIALS',
       'AUTH::EXPERIENTIAL', 'AUTH::EXTERNAL', 'AUTH::FORUM',
       'AUTH::SOCIAL_EXPECTATIONS', 'EXAMPLE', 'FIRST_TURN', 'LAST_TURN',
       'NUM_TURNS', 'PERS_PRON::PLUR',

In [3]:
data_roles = data[np.isfinite(data['TargetRole_0'])]
print(len(data_roles))
role_threads = list(set(zip(data_roles['article'], data_roles['thread'])))
# n = 0
# t = role_threads[n]
# t

21328


In [4]:
pd.set_option('display.max_colwidth', 999)

In [7]:
# Split based on conversation size
threads_bysize = {}

for i in range(2,5):
    sel = data_roles[data_roles['#editors']==i]
    threads_bysize[i] = list(set(zip(sel['article'], sel['thread'])))
    
sel = data_roles[data_roles['#editors']>=5]
threads_bysize[5] = list(set(zip(sel['article'], sel['thread'])))

df_samples = {}

sel_cols = ['article', 'thread', 'editor', '#editors', 'editor_talk'] + ['TargetRole_{}'.format(i) for i in range(5)]
#     + ['NontargetRole_{}'.format(i) for i in range(5)]
for i in range(2,6):
    s = random.sample(threads_bysize[i], 10)
    mask = list(map(lambda tup: tup in s, zip(data['article'], data['thread'])))
    df_samples[i] = data[mask].loc[:, sel_cols]

In [8]:
df_samples[2]

,article,thread,editor,#editors,editor_talk,TargetRole_0,TargetRole_1,TargetRole_2,TargetRole_3,TargetRole_4,NontargetRole_0,NontargetRole_1,NontargetRole_2,NontargetRole_3,NontargetRole_4
2141,Reichskonkordat,The passing of the Enabling Act,Str1977,2,"Ualtmann, you are mistaken about that: there was immunity for deputies (unless of course, when caught in the act) but this immunity is no right of the individual deputy (still isn't) but a measure to protect parliament. Hence it can be waived. Göring also didn't (and it wasn't him alone) arrest all SPDers but only some of them. All Communists however were arrested. A trick devised by the SPD to render parliament unable to vote was foiled by changing the procedure, so the remaining SPDers attened after all. It is correct to say that the the Centre Party's votes (and the BVP's) were necessary for the act to pass, in addition to those of Nazis, DNVP, both Liberal parties, and the smaller groupings. However, there is no foundation for including the Vatican in this. The Centre was not the Pope's puppet. (smile back) 16:39, 5 September 2006 (UTC)",0.024160,0.029535,0.028935,0.016705,0.900665,0.024740,0.017565,0.014296,0.011875,0.931523
2142,Reichskonkordat,The passing of the Enabling Act,UAltmann,2,"The Enabling Act could not have been passed with the votes of the Deutschnationale Partei, the Nazis and the Center Party alone. Decisive were two more circumstances: Herman Göring was Secretary of State of Prussia, thus he was commander in chief for the police. The other fact was that there was yet no immunity for parliament representatives. So, Hermann Göring assisted the passing of the enabling act by suddenly imprisoning almost all representatives of the Social Democratic Party (for whatever reasons) shortly before the Enabling Act was brought in. As two thirds of the representatives attending were necessary, the Act could then be passed. I do not have a source for this, this belongs to common knowledge. So, if the article tells about the Enabling Act putting up the (adventurous) theory that the vatican helped the Enabling Act in turn for the Reichskonkordat, it should as well mention these facts above, which brings the true circumstances, especially the limited powers of the ...",0.017008,0.018641,0.010611,0.010285,0.943455,0.023726,0.025460,0.028469,0.014524,0.907820
4165,Vladimir Sapovskiy,Basic information about Sekret Uspekha and winners,ADimiTR,2,"This show hadn't been popular in Russia and didn't have great ratings, compared with competitors like Fabrika Zvyozd (russian Star Academy). The winners have already forgotten, so it 's difficult to find any information about them. By the way, it'll be a new season of this show called Faktor A, produced by Alla Pugacheva, based on popularity of Ukrainian version. (talk) 13:43, 29 March 2011 (UTC)",0.383358,0.193347,0.131925,0.066272,0.225098,0.508092,0.133787,0.216416,0.062798,0.078907
4166,Vladimir Sapovskiy,Basic information about Sekret Uspekha and winners,Werldwayd,2,"We need help from a Russian contributor who can tell us about the seasons of Sekret Uspekha and the winners, Vladimir Sapovskiy (Season 1 , 2005) and Nikolai Timohin (Season 2, 2007). I coudln't find any page either of the two nor a page for Sekret Uspekha itself on the Russian Wikipedia and all the videos on the YouTube channel are for non-winners. werldwayd (talk) 05:09, 11 January 2010 (UTC)",0.491981,0.099252,0.236130,0.066609,0.106028,0.391163,0.184060,0.122582,0.045436,0.256760
6083,Anti-abortion violence,"all violence should be condemned, so acknowledge existence of violence against peaceful pro-life people",Binksternet,2,"The attacks on pro-life demonstrators are of vastly less importance than the attacks on pro-choice people and property. What we are doing here in this article is presenting the correct balance, that is, the great preponderance of violence by pro-life people. (talk) 21:24, 6 July 2012 (UTC) :::Ad hominem swipes are not going to help the argumen

In [10]:
df_samples[4]

,article,thread,editor,#editors,editor_talk,TargetRole_0,TargetRole_1,TargetRole_2,TargetRole_3,TargetRole_4,NontargetRole_0,NontargetRole_1,NontargetRole_2,NontargetRole_3,NontargetRole_4
1878,Classical Chinese medicine,Pseudoscience,Herbxue,4,": Nope - it is clearer. Anything more is OR. (talk) 23:11, 27 April 2014 (UTC) ::::::::: Definitely not pointless, this matter is not settled.Herbxue (talk) 17:14, 30 April 2014 (UTC)",0.082101,0.099922,0.661842,0.079122,0.077012,0.178038,0.131765,0.391142,0.161894,0.137161
1879,Classical Chinese medicine,Pseudoscience,JzG,4,"Feel free to suggest a better wording of that sentence. Guy (Help!) 09:50, 27 April 2014 (UTC)",0.374325,0.207561,0.124756,0.127467,0.165891,0.095452,0.061256,0.583400,0.158996,0.100896
1880,Classical Chinese medicine,Pseudoscience,Mallexikon,4,"For the pseudoscience label, we're currently using this source: ""So if traditional Chinese medicine is so great, why hasn't the qualitative study of its outcomes opened the door to a flood of cures? The most obvious answer is that it actually has little to offer: it is largely just pseudoscience, with no rational mechanism of action for most of its therapies."" This sentence is a grammatically difficult structure, and I'd like to point out that the colon connects the ""it is largely just pseudoscience"" with the ""the most obvious answer is..."". That means, this article is only suspecting that (the reason why success stories like the discovery of Artemisin are so few and far between is that) TCM is a pseudoscience. (talk) 01:50, 27 April 2014 (UTC) ::This source has been correctly reflected in our article's ""Drug research"" section: ""... a 2007 editorial in Nature said that while this may simply be because TCM is largely pseudoscience..."" ::The sentence featured in our lede (""... it is...",0.023765,0.012738,0.649922,0.232076,0.081498,0.172338,0.162157,0.443387,0.097162,0.124955
1881,Classical Chinese medicine,Pseudoscience,QuackGuru,4,"The suggestion to state only ""TCM has also been labeled pseudoscience"". in the lede does not tell the reader much. The previous wording added by User:Mallexikon was not an improvement. For example, it has been suspected is weasel wordings to skew/dilute the facts. (talk) 17:00, 27 April 2014 (UTC)",0.133022,0.103877,0.475118,0.135457,0.152527,0.179280,0.116011,0.492575,0.120337,0.091797
2473,Palomino Creek,Country of Rockstar North,Benboy00,4,"From what I can see, it doesn't unnecessarily complicate anything, and it adds some information. The country of development isnt actually mentioned anywhere else in the article, which is a bit strange, even though it is in that category. If there isnt a problem having something there, and it adds information, why remove it? The fact that rockstar games is american and rockstar north is british is probably new to many people, especially as all (or almost all) of the GTA series are set in america. (talk) 15:06, 2 November 2013 (UTC) You don't seem to have really addressed my points. It doesn't negatively affect the readability of the article, and does not significantly increase the length. It is an interesting piece of information, for the reasons I stated before, and it looks like that is a good place for it. I also notice that you removed the content before editing this talk page. By all means feel free to add it to the games series page as well if you want. (talk) 12:49, 3 Novemb...",0.189303,0.235078,0.209947,0.016561,0.349111,0.136346,0.455519,0.070734,0.236655,0.100747
2474,Palomino Creek,Country of Rockstar North,Hellknowz,4,"This is probably not significant enough to be mentioned in lead, at least we usually don't as it just clutters already long intro sentences. However, I see no reason why a few words (e.g. ""UK-based developer"") about the developer's location can't be added to the development section. — HELLKNOWZ ▎TALK 15:40, 3 November 2013 (UTC)",0.155008,0.628868,0.088045,0.054798,0.073282,0.175488,0.301663,0.138750,0.203757,0.1803

In [16]:
sel_cols = ['article', 'thread', 'editor', '#editors', 'editor_talk'] + ['TargetRole_{}'.format(i) for i in range(5)] + \
    ['NontargetRole_{}'.format(i) for i in range(5)]
selected = data[(data['article']==t[0]) & (data['thread']==t[1])].loc[:, sel_cols]
selected

,article,thread,editor,#editors,editor_talk,TargetRole_0,TargetRole_1,TargetRole_2,TargetRole_3,TargetRole_4,NontargetRole_0,NontargetRole_1,NontargetRole_2,NontargetRole_3,NontargetRole_4
40465,Bender Gets Made,Bendfellas?,75pickup,2,"I do own the DVD, it lists it as ""Bender Gets Made"", so I'm moving the article back. TEMPLATE[User, 75pickup] 23:12, 1 February 2007 (UTC)",0.174999,0.532844,0.077803,0.106393,0.107961,0.496822,0.13369,0.242966,0.081012,0.045510
40466,Bender Gets Made,Bendfellas?,Stardust8212,2,"Regarding the recent move: Could someone please verify, preferably from the DVD box, that ""Bender gets Made (a.k.a. Bendfellas)"" is the true, full, correct title? With all the discussion of the title over at ""The Series Has Landed"" regarding its title I am surprised this hadn't been brought up before. I hadn't seen this anywhere before but I do not own Season 2. GotFuturama.com and Amazon both list it as ""Bender Gets Made"". 20:37, 25 January 2007 (UTC)",0.579214,0.119772,0.218283,0.049738,0.032993,0.208607,0.47091,0.118863,0.099572,0.102047


# Look into some dev and test rows not having role distros

In [16]:
devtest = data[data['split'].isin(['dev', 'test'])]
print(len(devtest))
devtest_threads = list(set(zip(devtest['article'], devtest['thread'])))
n = 0
t = devtest_threads[n]
t

21180


('Multiplayer online casual game', 'Article Duplication?')

In [13]:
sel_cols = ['split', 'article', 'thread', 'editor', '#editors', 'editor_talk'] + ['TargetRole_{}'.format(i) for i in range(5)] + \
    ['NontargetRole_{}'.format(i) for i in range(5)]
selected = data[(data['article']==t[0]) & (data['thread']==t[1])].loc[:, sel_cols]
selected

,split,article,thread,editor,#editors,editor_talk,TargetRole_0,TargetRole_1,TargetRole_2,TargetRole_3,TargetRole_4,NontargetRole_0,NontargetRole_1,NontargetRole_2,NontargetRole_3,NontargetRole_4
37384,dev,Multiplayer online casual game,Article Duplication?,Fyrn,3,The definition within Wikipedia is pretty inex...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37385,dev,Multiplayer online casual game,Article Duplication?,Jeraphine Gryphon,3,":: Half of my DND group didn't roleplay, and n...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37386,dev,Multiplayer online casual game,Article Duplication?,Mrwojo,3,"No. MMORPGs are role-playing games, whereas MM...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
devtest['TargetRole_0'].count()

10650

In [28]:
eg = dev.loc[0]
eg

Unnamed: 0                                                                      0
article                                                          Attic Insulation
thread                                    Topics, what a radiant barrier touches.
editor                                                                   Ccrrccrr
editor_talk                     The "cold climate" and "hot climate" sections ...
other_talk                      :You are right that any contact made will nega...
#editor_turns                                                                   1
#other_turns                                                                    2
editor_score                                                             0.433265
fold                                                                            9
ftopic0                                                                         4
ftopic1                                                                         0
ftopic2         

In [31]:
# devtest[(devtest['article']==eg['article']) & (devtest['thread']==eg['thread']) & (devtest['editor']==eg['editor'])]
# devtest[(devtest['article']==eg['article']) & (devtest['thread']==eg['thread'])]
devtest[(devtest['article']==eg['article'])]

,split,fold,article,thread,editor,#editors,editor_talk,other_talk,#editor_turns,#other_turns,...,TargetRole_1,TargetRole_2,TargetRole_3,TargetRole_4,NontargetRole_0,NontargetRole_1,NontargetRole_2,NontargetRole_3,NontargetRole_4,editor_score


In [35]:
# Make sure all dev keys present in combo dataset
combo_dev = devtest[devtest['split']=='dev']
combined_keys = sorted(set(zip(combo_dev['article'], combo_dev['thread'], combo_dev['editor'])))
print(len(combined_keys))

dev_keys = sorted(set(zip(dev['article'], dev['thread'], dev['editor'])))
print(len(dev_keys))
# extra_dev = dev_keys.intersection(combined_keys)
# all(k in combined_keys  dev_keys)

print(combined_keys[:10])
print(dev_keys[:10])

10530
10678
[('-ana', 'Origin', 'Charles Matthews'), ('-ana', 'Origin', 'JackofOz'), ('007: From Russia with Love', 'Title', 'Emperor001'), ('007: From Russia with Love', 'Title', 'K1Bond007'), ('1829 braille', 'Table', 'Kwamikagami'), ('1829 braille', 'Table', 'Vanisaac'), ('1914-1918', 'Orthographic Correction', 'Older and ... well older'), ('1914-1918', 'Orthographic Correction', 'Three-quarter-ten'), ('1914-1918', 'Peace treaties', 'Blaylockjam10'), ('1914-1918', 'Peace treaties', 'Rjensen')]
[('"V" Is for Vagina', 'Why is the "i" capitalized?', 'Gunmetal Angel'), ('"V" Is for Vagina', 'Why is the "i" capitalized?', 'Jennavecia'), ("(I Can't Get No) Satisfaction", 'Britney Spears cover', 'Edgarde'), ("(I Can't Get No) Satisfaction", 'Britney Spears cover', 'Phiwum'), ("(I Can't Get No) Satisfaction", 'Britney Spears cover', 'Woodywoodpeckerthe3rd'), ("(I Can't Get No) Satisfaction", 'Covers and esp. singles', 'John Cardinal'), ("(I Can't Get No) Satisfaction", 'Covers and esp. sing

In [42]:
# Make sure all test keys present in combo dataset
combo_test = devtest[devtest['split']=='test']
# combined_keys = sorted(set(zip(combo_test['article'], combo_test['thread'], combo_test['editor'])))
combined_keys = set(zip(combo_test['article'], combo_test['thread'], combo_test['editor']))
print(len(combined_keys))

# test_keys = sorted(set(zip(test['article'], test['thread'], test['editor'])))
test_keys = set(zip(test['article'], test['thread'], test['editor']))
print(len(test_keys))
inter = test_keys.intersection(combined_keys)
# all(k in combined_keys  dev_keys)
print(len(inter))

# print(combined_keys[:10])
# print(dev_keys[:10])

10650
10650
10650


In [36]:
devtest

,split,fold,article,thread,editor,#editors,editor_talk,other_talk,#editor_turns,#other_turns,...,TargetRole_1,TargetRole_2,TargetRole_3,TargetRole_4,NontargetRole_0,NontargetRole_1,NontargetRole_2,NontargetRole_3,NontargetRole_4,editor_score
4,test,0,World's tallest Ferris wheels,The London Eye *isn’t* a Ferris Wheel!,JlACEer,2,:The trend among educated reporters is to refe...,"This article is very confused, as it insists o...",1,3,...,0.172396,0.232299,0.131135,0.372624,0.144519,0.103481,0.449360,0.088397,0.214243,0.850000
5,test,0,World's tallest Ferris wheels,The London Eye *isn’t* a Ferris Wheel!,LtPowers,2,"While you are entitled to your opinion here, y...","This article is very confused, as it insists o...",1,3,...,0.095248,0.477221,0.129327,0.183696,0.073462,0.169057,0.248033,0.112938,0.396509,1.000000
6,dev,2,Storyline of the Mario series,Reference In Assasin's Creed 2,09aidepikiw,2,"This only seems to be a small reference, but i...","I would say ""No"" but...what do you think? comp...",1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.630479
7,dev,2,Storyline of the Mario series,Reference In Assasin's Creed 2,Wizardman,2,"I don't think it's noteworthy in this article,...","This only seems to be a small reference, but i...",1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.288640
8,dev,3,Libyan Arab Socialist Ba'ath Party,Trimming lead,BoogaLouie,2,I have trimmed the lead to eliminate repetitio...,"Umm, someone put ""rednex"" and ""reddya"" in the ...",1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.830882
9,dev,3,Libyan Arab Socialist Ba'ath Party,Trimming lead,Lihaas,2,"Umm, someone put ""rednex"" and ""reddya"" in the ...",I have trimmed the lead to eliminate repetitio...,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.500000
10,dev,2,Guglielmo Marconi,Marconi's British patents,Blainster,2,"A couple of sources [ ] and [ ], list this pat...",: I beleve that the early British patents had ...,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.972973
11,dev,2,Guglielmo Marconi,Marconi's British patents,Edison,2,: I beleve that the early British patents had ...,"A couple of sources [ ] and [ ], list this pat...",1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
12,test,1,September 2007 in sports,Oussama Mellouli - first Arab world champion ...,Jmlk17,2,"Seems like a good idea. 10:18, 12 September 20...",This happened 11 September 2007. See biogram O...,1,1,...,0.513305,0.068026,0.074202,0.120464,0.340391,0.552415,0.035009,0.023081,0.049104,0.595376
13,test,1,September 2007 in sports,Oussama Mellouli - first Arab world champion ...,Mareklug,2,This happened 11 September 2007. See biogram O...,"Seems like a good idea. 10:18, 12 September 20...",1,1,...,0.522063,0.041852,0.029810,0.048242,0.252684,0.485220,0.102363,0.066662,0.093071,0.852941


In [37]:
data[data['article']=='1-800-FREE-411']

,split,fold,article,thread,editor,#editors,editor_talk,other_talk,#editor_turns,#other_turns,...,TargetRole_1,TargetRole_2,TargetRole_3,TargetRole_4,NontargetRole_0,NontargetRole_1,NontargetRole_2,NontargetRole_3,NontargetRole_4,editor_score
6297,train,9,1-800-FREE-411,Advertising?,Mattnad,2,The introduction to this article appears to be...,"Indeed, the article is much less of a promo pi...",2,1,...,0.013884,0.738084,0.015685,0.169577,0.037747,0.115787,0.702478,0.061766,0.082223,0.825342
6298,train,9,1-800-FREE-411,Advertising?,Retro00064,2,"Indeed, the article is much less of a promo pi...",The introduction to this article appears to be...,1,2,...,0.070850,0.729912,0.052668,0.072756,0.086734,0.013061,0.713274,0.025612,0.161318,0.846154


In [39]:
# Double-check fold info
fold_info = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/fold_info.tsv', sep='\t', header=None)
fold_info[fold_info[0]=='1-800-FREE-411']

,0,1
2689,1-800-FREE-411,9


In [40]:
dev['fold'].unique()

array([9, 8])

In [41]:
test['fold'].unique()

array([0, 1])

In [33]:
# Make sure all dev keys present in combo dataset
combined_keys = set(zip(devtest['article'], devtest['thread'], devtest['editor']))
len(combined_keys)

test_keys = set(zip(test['article'], test['thread'], test['editor']))
extra_dev = dev_keys.intersection(combined_keys)
# all(k in combined_keys  dev_keys)
len(extra_dev)

0

In [17]:
dev = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/role_weights/PRPM_K5-dev_all.csv')
print(len(dev))

10678


In [22]:
test = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/role_weights/PRPM_K5-test_all.csv')
print(len(test))

10650


In [23]:
len(test) + len(dev)

21328

In [24]:
dev['TargetRole_0'].count()

10678

In [25]:
test['TargetRole_0'].count()

10650

In [26]:
data['TargetRole_0'].count()

21328

In [44]:
data_roles = data[np.isfinite(data['TargetRole_0'])]
data_roles['fold'].unique()

array([0, 1, 9, 8])

# Normalize features for Carolyn

## Merge in PRPM 5-role weights

In [2]:
dev = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/role_weights/PRPM_K5-dev_all.csv')
print(len(dev))
test = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/role_weights/PRPM_K5-test_all.csv')
print(len(test))

10678
10650


In [3]:
merged = pd.merge(dev, test, how='outer')
print(len(merged))

21328


In [4]:
merged.columns.tolist()

['Unnamed: 0',
 'article',
 'thread',
 'editor',
 'editor_talk',
 'other_talk',
 '#editor_turns',
 '#other_turns',
 'editor_score',
 'fold',
 'ftopic0',
 'ftopic1',
 'ftopic2',
 'ftopic3',
 'ftopic4',
 'ftopic5',
 'ftopic6',
 'ftopic7',
 'ftopic8',
 'ftopic9',
 'ftopic10',
 'ftopic11',
 'ftopic12',
 'ftopic13',
 'ftopic14',
 'ftopic15',
 'ftopic16',
 'ftopic17',
 'ftopic18',
 'ftopic19',
 'other_ftopic0',
 'other_ftopic1',
 'other_ftopic2',
 'other_ftopic3',
 'other_ftopic4',
 'other_ftopic5',
 'other_ftopic6',
 'other_ftopic7',
 'other_ftopic8',
 'other_ftopic9',
 'other_ftopic10',
 'other_ftopic11',
 'other_ftopic12',
 'other_ftopic13',
 'other_ftopic14',
 'other_ftopic15',
 'other_ftopic16',
 'other_ftopic17',
 'other_ftopic18',
 'other_ftopic19',
 '#editors',
 'ed_dialog_act_pred',
 'ed_pred',
 'all_dialog_act_pred',
 'all_pred',
 'ART::DEF',
 'ART::INDEF',
 'AUTH::CREDENTIALS',
 'AUTH::EXPERIENTIAL',
 'AUTH::EXTERNAL',
 'AUTH::FORUM',
 'AUTH::SOCIAL_EXPECTATIONS',
 'EXAMPLE',
 'FI

In [5]:
feats = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_allfeats.csv')

In [8]:
allmerged = pd.merge(feats, merged, how='left')
len(allmerged)

53175

In [10]:
selected = allmerged[feats.columns.tolist() + ['TargetRole_{}'.format(i) for i in range(5)] + ['NontargetRole_{}'.format(i) for i in range(5)]]
selected.columns

Index(['split', 'fold', 'article', 'thread', 'editor', '#editors',
       'editor_talk', 'other_talk', '#editor_turns', '#other_turns', 'ftopic0',
       'ftopic1', 'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6',
       'ftopic7', 'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12',
       'ftopic13', 'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18',
       'ftopic19', 'other_ftopic0', 'other_ftopic1', 'other_ftopic2',
       'other_ftopic3', 'other_ftopic4', 'other_ftopic5', 'other_ftopic6',
       'other_ftopic7', 'other_ftopic8', 'other_ftopic9', 'other_ftopic10',
       'other_ftopic11', 'other_ftopic12', 'other_ftopic13', 'other_ftopic14',
       'other_ftopic15', 'other_ftopic16', 'other_ftopic17', 'other_ftopic18',
       'other_ftopic19', 'ART::DEF', 'ART::INDEF', 'AUTH::CREDENTIALS',
       'AUTH::EXPERIENTIAL', 'AUTH::EXTERNAL', 'AUTH::FORUM',
       'AUTH::SOCIAL_EXPECTATIONS', 'EXAMPLE', 'FIRST_TURN', 'LAST_TURN',
       'NUM_TURNS', 'PERS_PRON::PLUR',

In [11]:
cols = selected.columns.tolist()
new_cols = cols[:cols.index('editor_score')] + cols[cols.index('editor_score')+1:] + ['editor_score']
new_cols

['split',
 'fold',
 'article',
 'thread',
 'editor',
 '#editors',
 'editor_talk',
 'other_talk',
 '#editor_turns',
 '#other_turns',
 'ftopic0',
 'ftopic1',
 'ftopic2',
 'ftopic3',
 'ftopic4',
 'ftopic5',
 'ftopic6',
 'ftopic7',
 'ftopic8',
 'ftopic9',
 'ftopic10',
 'ftopic11',
 'ftopic12',
 'ftopic13',
 'ftopic14',
 'ftopic15',
 'ftopic16',
 'ftopic17',
 'ftopic18',
 'ftopic19',
 'other_ftopic0',
 'other_ftopic1',
 'other_ftopic2',
 'other_ftopic3',
 'other_ftopic4',
 'other_ftopic5',
 'other_ftopic6',
 'other_ftopic7',
 'other_ftopic8',
 'other_ftopic9',
 'other_ftopic10',
 'other_ftopic11',
 'other_ftopic12',
 'other_ftopic13',
 'other_ftopic14',
 'other_ftopic15',
 'other_ftopic16',
 'other_ftopic17',
 'other_ftopic18',
 'other_ftopic19',
 'ART::DEF',
 'ART::INDEF',
 'AUTH::CREDENTIALS',
 'AUTH::EXPERIENTIAL',
 'AUTH::EXTERNAL',
 'AUTH::FORUM',
 'AUTH::SOCIAL_EXPECTATIONS',
 'EXAMPLE',
 'FIRST_TURN',
 'LAST_TURN',
 'NUM_TURNS',
 'PERS_PRON::PLUR',
 'PERS_PRON::SING',
 'QUESTION',
 '

In [12]:
selected = selected[new_cols]

In [13]:
selected['TargetRole_2'].count()

21328

In [14]:
selected.to_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_scores_roles.csv', index=False)

## Rename allfeats columns, create normalized features

In [2]:
feats.columns

Index(['article', 'thread', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score', 'fold', 'ftopic0',
       'ftopic1', 'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6',
       'ftopic7', 'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12',
       'ftopic13', 'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18',
       'ftopic19', 'other_ftopic0', 'other_ftopic1', 'other_ftopic2',
       'other_ftopic3', 'other_ftopic4', 'other_ftopic5', 'other_ftopic6',
       'other_ftopic7', 'other_ftopic8', 'other_ftopic9', 'other_ftopic10',
       'other_ftopic11', 'other_ftopic12', 'other_ftopic13', 'other_ftopic14',
       'other_ftopic15', 'other_ftopic16', 'other_ftopic17', 'other_ftopic18',
       'other_ftopic19', '#editors', 'ed_dialog_act_pred', 'ed_pred',
       'all_dialog_act_pred', 'all_pred', 'ART::DEF', 'ART::INDEF',
       'AUTH::CREDENTIALS', 'AUTH::EXPERIENTIAL', 'AUTH::EXTERNAL',
       'AUTH::FORUM', 'AUTH::SOCIAL_EXPECTATI

In [4]:
feats['talk_len'] = feats['editor_talk'].map(lambda x: len(x.split()))
feats['talk_len']

0          52
1         144
2          75
3          43
4          24
5          54
6          58
7          22
8          22
9          28
10         60
11         49
12         10
13         36
14        363
15        177
16        123
17        132
18        351
19         71
20         92
21         51
22        862
23        112
24         77
25        119
26        184
27         77
28        106
29         32
         ... 
53145      89
53146     114
53147      13
53148      52
53149      76
53150     160
53151     144
53152      42
53153     147
53154     261
53155      70
53156      89
53157      64
53158     111
53159     263
53160     183
53161     246
53162      63
53163      39
53164      35
53165      79
53166      48
53167     164
53168     293
53169     255
53170    5553
53171     461
53172    1185
53173      51
53174      30
Name: talk_len, dtype: int64

In [5]:
cols = list(feats.columns)
count_featnames = cols[cols.index('ART::DEF'):cols.index('URL')+1]
count_featnames

['ART::DEF',
 'ART::INDEF',
 'AUTH::CREDENTIALS',
 'AUTH::EXPERIENTIAL',
 'AUTH::EXTERNAL',
 'AUTH::FORUM',
 'AUTH::SOCIAL_EXPECTATIONS',
 'EXAMPLE',
 'FIRST_TURN',
 'LAST_TURN',
 'NUM_TURNS',
 'PERS_PRON::PLUR',
 'PERS_PRON::SING',
 'QUESTION',
 'SENTI::NEG',
 'SENTI::POS',
 'URL']

In [6]:
for col in count_featnames:
    feats['{}_norm'.format(col)] = feats[col]/feats['talk_len']

In [8]:
feats.columns

Index(['article', 'thread', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score', 'fold', 'ftopic0',
       'ftopic1', 'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6',
       'ftopic7', 'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12',
       'ftopic13', 'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18',
       'ftopic19', 'other_ftopic0', 'other_ftopic1', 'other_ftopic2',
       'other_ftopic3', 'other_ftopic4', 'other_ftopic5', 'other_ftopic6',
       'other_ftopic7', 'other_ftopic8', 'other_ftopic9', 'other_ftopic10',
       'other_ftopic11', 'other_ftopic12', 'other_ftopic13', 'other_ftopic14',
       'other_ftopic15', 'other_ftopic16', 'other_ftopic17', 'other_ftopic18',
       'other_ftopic19', '#editors', 'ed_dialog_act_pred', 'ed_pred',
       'all_dialog_act_pred', 'all_pred', 'ART::DEF', 'ART::INDEF',
       'AUTH::CREDENTIALS', 'AUTH::EXPERIENTIAL', 'AUTH::EXTERNAL',
       'AUTH::FORUM', 'AUTH::SOCIAL_EXPECTATI

In [9]:
feats['ed_old_pred'].count()

0

In [10]:
feats['ed_ftopic_pred'].count()

0

In [11]:
feats['all_nonbow_pred'].count()

0

In [12]:
feats['all_pred'].count()

10650

In [13]:
for col in ['ed_old_pred', 'ed_ftopic_pred', 'all_nonbow_pred', 'all_pred']:
    feats.drop(col, axis=1, inplace=True)

In [14]:
feats.columns

Index(['article', 'thread', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score', 'fold', 'ftopic0',
       'ftopic1', 'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6',
       'ftopic7', 'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12',
       'ftopic13', 'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18',
       'ftopic19', 'other_ftopic0', 'other_ftopic1', 'other_ftopic2',
       'other_ftopic3', 'other_ftopic4', 'other_ftopic5', 'other_ftopic6',
       'other_ftopic7', 'other_ftopic8', 'other_ftopic9', 'other_ftopic10',
       'other_ftopic11', 'other_ftopic12', 'other_ftopic13', 'other_ftopic14',
       'other_ftopic15', 'other_ftopic16', 'other_ftopic17', 'other_ftopic18',
       'other_ftopic19', '#editors', 'ed_dialog_act_pred', 'ed_pred',
       'all_dialog_act_pred', 'ART::DEF', 'ART::INDEF', 'AUTH::CREDENTIALS',
       'AUTH::EXPERIENTIAL', 'AUTH::EXTERNAL', 'AUTH::FORUM',
       'AUTH::SOCIAL_EXPECTATIONS', 'EXAMP

In [15]:
feats['ed_pred'].count()

10650

In [17]:
feats.drop('ed_pred', axis=1, inplace=True)

In [18]:
feats.drop('ed_dialog_act_pred', axis=1, inplace=True)

In [20]:
feats.drop('all_dialog_act_pred', axis=1, inplace=True)

In [16]:
feats['ed_dialog_act_pred'].count()

10650

In [21]:
feats.columns

Index(['article', 'thread', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score', 'fold', 'ftopic0',
       'ftopic1', 'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6',
       'ftopic7', 'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12',
       'ftopic13', 'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18',
       'ftopic19', 'other_ftopic0', 'other_ftopic1', 'other_ftopic2',
       'other_ftopic3', 'other_ftopic4', 'other_ftopic5', 'other_ftopic6',
       'other_ftopic7', 'other_ftopic8', 'other_ftopic9', 'other_ftopic10',
       'other_ftopic11', 'other_ftopic12', 'other_ftopic13', 'other_ftopic14',
       'other_ftopic15', 'other_ftopic16', 'other_ftopic17', 'other_ftopic18',
       'other_ftopic19', '#editors', 'ART::DEF', 'ART::INDEF',
       'AUTH::CREDENTIALS', 'AUTH::EXPERIENTIAL', 'AUTH::EXTERNAL',
       'AUTH::FORUM', 'AUTH::SOCIAL_EXPECTATIONS', 'EXAMPLE', 'FIRST_TURN',
       'LAST_TURN', 'NUM_TURNS', 'PERS_PRON:

In [24]:
cols = feats.columns.tolist()
new_cols = ['split', 'fold'] + cols[:cols.index('editor')+1] + ['#editors'] + \
            cols[cols.index('editor_talk'):cols.index('#other_turns')+1] + \
            cols[cols.index('ftopic0'):cols.index('other_ftopic19')+1] + \
            cols[cols.index('ART::DEF'):cols.index('URL')+1] + \
            cols[cols.index('ART::DEF_norm'):] + ['talk_len', 'editor_score'] 
new_cols

['split',
 'fold',
 'article',
 'thread',
 'editor',
 '#editors',
 'editor_talk',
 'other_talk',
 '#editor_turns',
 '#other_turns',
 'ftopic0',
 'ftopic1',
 'ftopic2',
 'ftopic3',
 'ftopic4',
 'ftopic5',
 'ftopic6',
 'ftopic7',
 'ftopic8',
 'ftopic9',
 'ftopic10',
 'ftopic11',
 'ftopic12',
 'ftopic13',
 'ftopic14',
 'ftopic15',
 'ftopic16',
 'ftopic17',
 'ftopic18',
 'ftopic19',
 'other_ftopic0',
 'other_ftopic1',
 'other_ftopic2',
 'other_ftopic3',
 'other_ftopic4',
 'other_ftopic5',
 'other_ftopic6',
 'other_ftopic7',
 'other_ftopic8',
 'other_ftopic9',
 'other_ftopic10',
 'other_ftopic11',
 'other_ftopic12',
 'other_ftopic13',
 'other_ftopic14',
 'other_ftopic15',
 'other_ftopic16',
 'other_ftopic17',
 'other_ftopic18',
 'other_ftopic19',
 'ART::DEF',
 'ART::INDEF',
 'AUTH::CREDENTIALS',
 'AUTH::EXPERIENTIAL',
 'AUTH::EXTERNAL',
 'AUTH::FORUM',
 'AUTH::SOCIAL_EXPECTATIONS',
 'EXAMPLE',
 'FIRST_TURN',
 'LAST_TURN',
 'NUM_TURNS',
 'PERS_PRON::PLUR',
 'PERS_PRON::SING',
 'QUESTION',
 '

In [25]:
print(len(cols))
print(len(new_cols))

86
86


In [26]:
feats = feats[new_cols]
feats.columns

Index(['split', 'fold', 'article', 'thread', 'editor', '#editors',
       'editor_talk', 'other_talk', '#editor_turns', '#other_turns', 'ftopic0',
       'ftopic1', 'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6',
       'ftopic7', 'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12',
       'ftopic13', 'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18',
       'ftopic19', 'other_ftopic0', 'other_ftopic1', 'other_ftopic2',
       'other_ftopic3', 'other_ftopic4', 'other_ftopic5', 'other_ftopic6',
       'other_ftopic7', 'other_ftopic8', 'other_ftopic9', 'other_ftopic10',
       'other_ftopic11', 'other_ftopic12', 'other_ftopic13', 'other_ftopic14',
       'other_ftopic15', 'other_ftopic16', 'other_ftopic17', 'other_ftopic18',
       'other_ftopic19', 'ART::DEF', 'ART::INDEF', 'AUTH::CREDENTIALS',
       'AUTH::EXPERIENTIAL', 'AUTH::EXTERNAL', 'AUTH::FORUM',
       'AUTH::SOCIAL_EXPECTATIONS', 'EXAMPLE', 'FIRST_TURN', 'LAST_TURN',
       'NUM_TURNS', 'PERS_PRON::PLUR',

In [27]:
feats.to_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_allfeats.csv', index=False)

# Get preds from Diyi's model, split by conversation size

In [15]:
feats = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_allfeats.csv')

n_roles = 5
with open('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/diyi_model/ijcnlp_{0}/result/preds_{0}.txt'.format(n_roles)) as f:
    lines = [x for x in f.read().splitlines()]
    preds = [float(x.split('\t')[0]) for x in lines]
    scores = [float(x.split('\t')[1]) for x in lines]
    
print(len(preds))

test = feats[feats['split']=='test']
test.columns

10650


Index(['article', 'thread', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score', 'fold', 'ftopic0',
       'ftopic1', 'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6',
       'ftopic7', 'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12',
       'ftopic13', 'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18',
       'ftopic19', 'other_ftopic0', 'other_ftopic1', 'other_ftopic2',
       'other_ftopic3', 'other_ftopic4', 'other_ftopic5', 'other_ftopic6',
       'other_ftopic7', 'other_ftopic8', 'other_ftopic9', 'other_ftopic10',
       'other_ftopic11', 'other_ftopic12', 'other_ftopic13', 'other_ftopic14',
       'other_ftopic15', 'other_ftopic16', 'other_ftopic17', 'other_ftopic18',
       'other_ftopic19', '#editors', 'ed_dialog_act_pred', 'ed_pred',
       'all_dialog_act_pred', 'all_pred', 'ART::DEF', 'ART::INDEF',
       'AUTH::CREDENTIALS', 'AUTH::EXPERIENTIAL', 'AUTH::EXTERNAL',
       'AUTH::FORUM', 'AUTH::SOCIAL_EXPECTATI

In [16]:
# Get Diyi alignment of test data
summary = '/home/michael/school/research/wp/wikipedia/data/talk/enwiki/diyi_model/ijcnlp_{}/summary_test.txt'.format(n_roles)
with open(summary) as f:
    lines = f.read().splitlines()[4:]
    
# parse lines
conv_ids = [line.split('\t') for line in lines]

diyi_scores = pd.DataFrame(conv_ids, columns=['article', 'thread', 'editor'])
diyi_scores['diyi_edscore'] = scores
diyi_scores['diyi_{}_pred'.format(n_roles)] = preds
diyi_scores

,article,thread,editor,diyi_edscore,diyi_5_pred
0,Anaerobic digestion,Could large scale anaerobic digestion reduce n...,Velella,0.394081,0.583047
1,Anaerobic digestion,Could large scale anaerobic digestion reduce n...,Vortexrealm,0.310470,0.581886
2,General War,Transwiki to Memory Alpha,Ravenhull,0.263305,0.534918
3,General War,Transwiki to Memory Alpha,Rindis,0.735129,0.550764
4,Georg Forster,Georg or George?,Carcharoth,1.000000,0.698320
5,Georg Forster,Georg or George?,Kusma,0.810828,0.669517
6,XBone,Aka Xbone,Masem,0.556943,0.553435
7,XBone,Aka Xbone,ThomasO1989,0.573427,0.507664
8,XBone,Aka Xbone,Sergecross73,0.377049,0.510012
9,XBone,Aka Xbone,Zero Serenity,0.703401,0.504603


In [17]:
merged = pd.merge(feats, diyi_scores)
print(len(merged))
print(len(test))
union = pd.merge(test, diyi_scores, how='outer')
print(len(union))
nomatch = union[union.isnull()['ftopic0']]
print(len(nomatch))
print(len(union) - len(test)) # good match

10650
10650
10650
0
0


In [18]:
# Check if RMSE correct (aligned predictions)--seems to be off for some reaso
np.sqrt(np.mean((merged['diyi_{}_pred'.format(n_roles)] - merged['editor_score']) ** 2))

0.31166436010821497

In [19]:
# Split by conversation size
for i in range(2,5):
    print(i, end=': ')
    selected = merged[merged['#editors']==i]
    print(np.sqrt(np.mean((selected['diyi_{}_pred'.format(n_roles)] - selected['editor_score']) ** 2)))
    
print("5+: ", end="")
selected = merged[merged['#editors']>=5]
print(np.sqrt(np.mean((selected['diyi_{}_pred'.format(n_roles)] - selected['editor_score']) ** 2)))

2: 0.308688095995
3: 0.315262415835
4: 0.307733001422
5+: 0.321212444193


## Investigate misalignment, old prediction merging

In [11]:
# Check if scores aligned
test['diyi_edscore']==test['editor_score']

4        False
5        False
12       False
13       False
22       False
23       False
24       False
25       False
26       False
27       False
72       False
73       False
86       False
87       False
88       False
89       False
98       False
99       False
120      False
121      False
128      False
129      False
130      False
131      False
136      False
137      False
138      False
139      False
151      False
152      False
         ...  
53033    False
53034    False
53035    False
53036    False
53037    False
53038    False
53039    False
53048    False
53049    False
53054    False
53055    False
53056    False
53057    False
53058    False
53059    False
53094    False
53095    False
53107    False
53108    False
53116    False
53117    False
53130    False
53131    False
53148    False
53149    False
53152    False
53153    False
53154    False
53173    False
53174    False
dtype: bool

In [6]:
# Check if RMSE correct (aligned predictions)--seems to be off for some reason
np.sqrt(np.mean((test['diyi_{}_pred'.format(n_roles)] - test['editor_score']) ** 2))

0.31725027564408148

In [53]:
merged = pd.merge(feats, test, how='left')
print(len(merged))
merged.columns

53175


Index(['article', 'thread', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score', 'fold', 'ftopic0',
       'ftopic1', 'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6',
       'ftopic7', 'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12',
       'ftopic13', 'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18',
       'ftopic19', 'other_ftopic0', 'other_ftopic1', 'other_ftopic2',
       'other_ftopic3', 'other_ftopic4', 'other_ftopic5', 'other_ftopic6',
       'other_ftopic7', 'other_ftopic8', 'other_ftopic9', 'other_ftopic10',
       'other_ftopic11', 'other_ftopic12', 'other_ftopic13', 'other_ftopic14',
       'other_ftopic15', 'other_ftopic16', 'other_ftopic17', 'other_ftopic18',
       'other_ftopic19', '#editors', 'ed_dialog_act_pred', 'ed_pred',
       'all_dialog_act_pred', 'all_pred', 'ART::DEF', 'ART::INDEF',
       'AUTH::CREDENTIALS', 'AUTH::EXPERIENTIAL', 'AUTH::EXTERNAL',
       'AUTH::FORUM', 'AUTH::SOCIAL_EXPECTATI

In [67]:
merged['diyi_2_pred'].count()

10650

In [73]:
# Split by conversation size

for i in range(2,5):
    print(i, end=': ')
    selected = test[test['#editors']==i]
    print(np.sqrt(np.mean((selected['diyi_2_pred'] - selected['editor_score']) ** 2)))
    
print("5: ", end="")
selected = test[test['#editors']>=5]
print(np.sqrt(np.mean((selected['diyi_2_pred'] - selected['editor_score']) ** 2)))

2: 0.313604605331
3: 0.319449537348
4: 0.310748722949
5:  0.319063778102


In [34]:
# Add in test, train, dev
fold_mapping = {0: 'test', 
                1: 'test',
                2: 'dev',
                3: 'dev',
                4: 'train',
                5: 'train',
                6: 'train',
                7: 'train',
                8: 'train',
                9: 'train',
               }

feats['split']=[fold_mapping[n] for n in feats['fold']]
feats['split']

0        train
1        train
2        train
3        train
4         test
5         test
6          dev
7          dev
8          dev
9          dev
10         dev
11         dev
12        test
13        test
14       train
15       train
16       train
17       train
18       train
19         dev
20         dev
21         dev
22        test
23        test
24        test
25        test
26        test
27        test
28       train
29       train
         ...  
53145    train
53146      dev
53147      dev
53148     test
53149     test
53150      dev
53151      dev
53152     test
53153     test
53154     test
53155      dev
53156      dev
53157    train
53158    train
53159    train
53160    train
53161      dev
53162      dev
53163    train
53164    train
53165      dev
53166      dev
53167      dev
53168      dev
53169      dev
53170      dev
53171      dev
53172      dev
53173     test
53174     test
Name: split, dtype: object

In [35]:
feats.to_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_allfeats.csv', index=False)

# Merge in role weight data

In [16]:
settings = ['dev', 'test']
n_roles = range(2,6)
role_wts = {c: {} for c in n_roles}

for c in n_roles:
    for s in settings:
        role_wts[c][s] = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/role_weights/PRPM_K{}-{}_theta.csv'.format(c,s))
        role_wts[c][s].rename(columns={'page': 'article', 'user': 'editor'}, inplace=True)

In [17]:
role_wts[2]['test']

,article,thread,editor,TargetRole_0,TargetRole_1,NontargetRole_0,NontargetRole_1
0,World's tallest Ferris wheels,The London Eye *isn’t* a Ferris Wheel!,JlACEer,0.364258,0.635742,0.600072,0.399928
1,World's tallest Ferris wheels,The London Eye *isn’t* a Ferris Wheel!,LtPowers,0.633458,0.366542,0.351651,0.648349
2,September 2007 in sports,Oussama Mellouli - first Arab world champion ...,Jmlk17,0.207151,0.792849,0.126375,0.873625
3,September 2007 in sports,Oussama Mellouli - first Arab world champion ...,Mareklug,0.131825,0.868175,0.229109,0.770891
4,Japanese acupuncture,Opening paragraph,BullRangifer,0.910830,0.089170,0.778856,0.221144
5,Japanese acupuncture,Opening paragraph,Herbxue,0.852633,0.147367,0.797472,0.202528
6,Japanese acupuncture,Opening paragraph,Jayaguru-Shishya,0.869727,0.130273,0.773198,0.226802
7,Japanese acupuncture,Opening paragraph,Jmh649,0.737491,0.262509,0.795567,0.204433
8,Japanese acupuncture,Opening paragraph,Klocek,0.730865,0.269135,0.799241,0.200759
9,Japanese acupuncture,Opening paragraph,QuackGuru,0.726102,0.273898,0.828110,0.171890


In [12]:
feats = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_allfeats.csv')
print(len(feats))
feats.columns

53175


Index(['article', 'thread', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score', 'fold', 'ftopic0',
       'ftopic1', 'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6',
       'ftopic7', 'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12',
       'ftopic13', 'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18',
       'ftopic19', 'other_ftopic0', 'other_ftopic1', 'other_ftopic2',
       'other_ftopic3', 'other_ftopic4', 'other_ftopic5', 'other_ftopic6',
       'other_ftopic7', 'other_ftopic8', 'other_ftopic9', 'other_ftopic10',
       'other_ftopic11', 'other_ftopic12', 'other_ftopic13', 'other_ftopic14',
       'other_ftopic15', 'other_ftopic16', 'other_ftopic17', 'other_ftopic18',
       'other_ftopic19', '#editors', 'ed_dialog_act_pred', 'ed_pred',
       'all_dialog_act_pred', 'all_pred', 'ART::DEF', 'ART::INDEF',
       'AUTH::CREDENTIALS', 'AUTH::EXPERIENTIAL', 'AUTH::EXTERNAL',
       'AUTH::FORUM', 'AUTH::SOCIAL_EXPECTATI

In [13]:
# Add in test, train, dev
fold_mapping = {0: 'test', 
                1: 'test',
                2: 'dev',
                3: 'dev',
                4: 'train',
                5: 'train',
                6: 'train',
                7: 'train',
                8: 'train',
                9: 'train',
               }

feats['split']=[fold_mapping[n] for n in feats['fold']]
feats['split']

0        train
1        train
2        train
3        train
4         test
5         test
6          dev
7          dev
8          dev
9          dev
10         dev
11         dev
12        test
13        test
14       train
15       train
16       train
17       train
18       train
19         dev
20         dev
21         dev
22        test
23        test
24        test
25        test
26        test
27        test
28       train
29       train
         ...  
53145    train
53146      dev
53147      dev
53148     test
53149     test
53150      dev
53151      dev
53152     test
53153     test
53154     test
53155      dev
53156      dev
53157    train
53158    train
53159    train
53160    train
53161      dev
53162      dev
53163    train
53164    train
53165      dev
53166      dev
53167      dev
53168      dev
53169      dev
53170      dev
53171      dev
53172      dev
53173     test
53174     test
Name: split, dtype: object

In [14]:
role_wts.keys()

dict_keys([2, 3, 4, 5])

In [23]:
merged =  {n: {} for n in n_roles}

for c in n_roles:
    for s in settings:
        merged[c][s] = pd.merge(feats, role_wts[c][s], how='inner')
        print(len(merged[c][s]))

10678
10650
10678
10650
10678
10650
10678
10650


In [31]:
for c in n_roles:
    for s in settings:
        print(len(role_wts[c][s]))

10678
10650
10678
10650
10678
10650
10678
10650


In [29]:
feats[np.isfinite(feats['ed_ftopic_pred'])]

,article,thread,editor,editor_talk,other_talk,#editor_turns,#other_turns,editor_score,fold,ftopic0,...,PERS_PRON::PLUR,PERS_PRON::SING,QUESTION,SENTI::NEG,SENTI::POS,URL,ed_old_pred,ed_ftopic_pred,all_nonbow_pred,split


In [24]:
merged[3]['test']

,article,thread,editor,editor_talk,other_talk,#editor_turns,#other_turns,editor_score,fold,ftopic0,...,ed_old_pred,ed_ftopic_pred,all_nonbow_pred,split,TargetRole_0,TargetRole_1,TargetRole_2,NontargetRole_0,NontargetRole_1,NontargetRole_2
0,World's tallest Ferris wheels,The London Eye *isn’t* a Ferris Wheel!,JlACEer,:The trend among educated reporters is to refe...,"This article is very confused, as it insists o...",1,3,0.850000,0,0.0,...,NaN,NaN,NaN,test,0.311426,0.262285,0.426288,0.220377,0.502682,0.276941
1,World's tallest Ferris wheels,The London Eye *isn’t* a Ferris Wheel!,LtPowers,"While you are entitled to your opinion here, y...","This article is very confused, as it insists o...",1,3,1.000000,0,0.0,...,NaN,NaN,NaN,test,0.288923,0.487081,0.223995,0.287322,0.290416,0.422262
2,September 2007 in sports,Oussama Mellouli - first Arab world champion ...,Jmlk17,"Seems like a good idea. 10:18, 12 September 20...",This happened 11 September 2007. See biogram O...,1,1,0.595376,1,0.0,...,NaN,NaN,NaN,test,0.741428,0.156251,0.102320,0.862105,0.076765,0.061130
3,September 2007 in sports,Oussama Mellouli - first Arab world champion ...,Mareklug,This happened 11 September 2007. See biogram O...,"Seems like a good idea. 10:18, 12 September 20...",1,1,0.852941,1,1.0,...,NaN,NaN,NaN,test,0.888333,0.058356,0.053310,0.742980,0.133959,0.123061
4,Japanese acupuncture,Opening paragraph,BullRangifer,"In an effort to resolve the neutral POV tag, I...",: I couldn't find anything wrong with the sour...,9,12,0.270586,1,4.0,...,NaN,NaN,NaN,test,0.056511,0.920095,0.023394,0.238840,0.615550,0.145609
5,Japanese acupuncture,Opening paragraph,Herbxue,::: Brangifer your complaints about this edito...,"In an effort to resolve the neutral POV tag, I...",4,17,0.098113,1,0.0,...,NaN,NaN,NaN,test,0.519850,0.422829,0.057322,0.168298,0.688340,0.143362
6,Japanese acupuncture,Opening paragraph,Jayaguru-Shishya,: I couldn't find anything wrong with the sour...,"In an effort to resolve the neutral POV tag, I...",1,20,0.096154,1,0.0,...,NaN,NaN,NaN,test,0.155525,0.792674,0.051801,0.252007,0.602886,0.145107
7,Japanese acupuncture,Opening paragraph,Jmh649,:Issues include: 1) use of non pubmed indexed ...,"In an effort to resolve the neutral POV tag, I...",2,19,0.536288,1,0.0,...,NaN,NaN,NaN,test,0.301921,0.613396,0.084682,0.196837,0.658831,0.144332
8,Japanese acupuncture,Opening paragraph,Klocek,"The ""properly sourced"" content I think you're ...","In an effort to resolve the neutral POV tag, I...",2,19,0.008219,1,0.0,...,NaN,NaN,NaN,test,0.063480,0.494944,0.441576,0.247006,0.675639,0.077355
9,Japanese acupuncture,Opening paragraph,QuackGuru,::::You have a lot of explaining to do. (talk)...,"In an effort to resolve the neutral POV tag, I...",3,18,0.407689,1,0.0,...,NaN,NaN,NaN,test,0.238853,0.697656,0.063491,0.220926,0.644402,0.134672


In [30]:
for c in n_roles:
    for s in settings:
        merged[c][s].to_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/PRPM_K{}-{}_all.csv'.format(c, s), index=False)

# Explicitly divide train, dev, test data sets

In [2]:
feats = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_allfeats.csv')
len(feats)

53175

In [3]:
test = feats[feats['fold'].isin([0,1])]
print(len(test))

dev = feats[feats['fold'].isin([2,3])]
print(len(dev))

train = feats[feats['fold'].isin(range(4,10))]
print(len(train))

10650
10530
31995


In [4]:
test.to_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/diyi_model/data/test.csv', index=False)
train.to_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/diyi_model/data/train.csv', index=False)
dev.to_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/diyi_model/data/dev.csv', index=False)

# Merge in old Yohan features

In [2]:
oldfeats = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_oldfeats.csv')
oldfeats

,article,thread_title,editor,ART::DEF,ART::INDEF,AUTH::CREDENTIALS,AUTH::EXPERIENTIAL,AUTH::EXTERNAL,AUTH::FORUM,AUTH::SOCIAL_EXPECTATIONS,EXAMPLE,FIRST_TURN,LAST_TURN,NUM_TURNS,PERS_PRON::PLUR,PERS_PRON::SING,QUESTION,SENTI::NEG,SENTI::POS,URL
0,"""Tiger"" William Dunlop",Current image,Deconstructhis,9.0,8.0,0.014329,0.006640,-0.000627,0.000763,0.002057,0.0,1,0,1.0,2.0,8.0,0.0,1.0,11.0,0.0
1,"""Tiger"" William Dunlop",Current image,Ukexpat,2.0,0.0,0.007279,0.000271,-0.000713,0.000431,-0.000244,0.0,0,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0
2,"""Tiger"" William Dunlop",Current image,Skinsmoke,0.0,0.0,0.010281,0.001874,-0.000402,0.000055,0.000090,0.0,0,0,1.0,0.0,1.0,0.0,0.0,2.0,0.0
3,"""Tiger"" William Dunlop",Requested move,Deconstructhis,6.0,8.0,0.041235,0.001973,-0.000304,0.001750,0.001725,0.0,0,0,2.0,2.0,4.0,0.0,0.0,14.0,0.0
4,"""Tiger"" William Dunlop",Requested move,CJ3370,0.0,2.0,-0.000023,0.000204,-0.000259,0.000075,0.000262,0.0,1,0,2.0,0.0,2.0,0.0,0.0,2.0,0.0
5,"""Tiger"" William Dunlop",Requested move,Skinsmoke,16.0,10.0,0.004528,0.004240,-0.001284,0.003148,0.001568,0.0,0,1,2.0,10.0,4.0,2.0,2.0,12.0,0.0
6,"""V"" Is for Vagina","Why is the ""i"" capitalized?",Gunmetal Angel,8.0,0.0,0.002190,0.001293,-0.000320,0.000743,0.000763,0.0,1,0,2.0,0.0,8.0,0.0,2.0,6.0,0.0
7,"""V"" Is for Vagina","Why is the ""i"" capitalized?",Jennavecia,34.0,10.0,0.026975,0.003796,-0.000345,0.000379,0.001230,0.0,0,1,2.0,4.0,18.0,0.0,4.0,14.0,0.0
8,&,"""O"" as a word",Saippuakauppias,0.0,1.0,0.005270,0.000335,0.000018,0.000030,0.000921,0.0,0,1,1.0,0.0,0.0,1.0,0.0,0.0,1.0
9,&,"""O"" as a word",Personman,0.0,0.0,-0.000014,0.000413,0.000067,0.000180,-0.000002,0.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
len(oldfeats)

69312

In [5]:
keith_datapts = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/instances.tsv', sep='\t', header=None, names=['article', 'thread', 'editor'])
keith_keys = set(zip(keith_datapts['article'], keith_datapts['thread'], keith_datapts['editor']))

# Check for Keith's keys
current = oldfeats
current_keys = set(zip(current['article'], current['thread_title'], current['editor']))
print(len(keith_keys.intersection(current_keys)))

53175


In [6]:
oldfeats.rename(columns={'thread_title': 'thread'}, inplace=True)
oldfeats.columns

Index(['article', 'thread', 'editor', 'ART::DEF', 'ART::INDEF',
       'AUTH::CREDENTIALS', 'AUTH::EXPERIENTIAL', 'AUTH::EXTERNAL',
       'AUTH::FORUM', 'AUTH::SOCIAL_EXPECTATIONS', 'EXAMPLE', 'FIRST_TURN',
       'LAST_TURN', 'NUM_TURNS', 'PERS_PRON::PLUR', 'PERS_PRON::SING',
       'QUESTION', 'SENTI::NEG', 'SENTI::POS', 'URL'],
      dtype='object')

In [7]:
feats = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_allfeats.csv')

In [9]:
merged = pd.merge(feats, oldfeats)
print(len(merged))
merged.columns

53175


Index(['article', 'thread', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score', 'fold', 'ftopic0',
       'ftopic1', 'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6',
       'ftopic7', 'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12',
       'ftopic13', 'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18',
       'ftopic19', 'other_ftopic0', 'other_ftopic1', 'other_ftopic2',
       'other_ftopic3', 'other_ftopic4', 'other_ftopic5', 'other_ftopic6',
       'other_ftopic7', 'other_ftopic8', 'other_ftopic9', 'other_ftopic10',
       'other_ftopic11', 'other_ftopic12', 'other_ftopic13', 'other_ftopic14',
       'other_ftopic15', 'other_ftopic16', 'other_ftopic17', 'other_ftopic18',
       'other_ftopic19', '#editors', 'ed_dialog_act_pred', 'ed_pred',
       'all_dialog_act_pred', 'all_pred', 'ART::DEF', 'ART::INDEF',
       'AUTH::CREDENTIALS', 'AUTH::EXPERIENTIAL', 'AUTH::EXTERNAL',
       'AUTH::FORUM', 'AUTH::SOCIAL_EXPECTATI

In [11]:
# Check for Keith's keys
current = merged
current_keys = set(zip(current['article'], current['thread'], current['editor']))
print(len(keith_keys.intersection(current_keys)))

53175


In [12]:
merged.to_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_allfeats.csv', index=False)

# Align dataset with Keith's instances

## Rebuild allfeats to regain missing # data

### Merge folds and ftopics

In [23]:
old_allfeats = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_allfeats.csv')
old_allfeats.columns

Index(['fold', 'article', 'thread', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score', 'ftopic0', 'ftopic1',
       'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6', 'ftopic7',
       'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12', 'ftopic13',
       'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18', 'ftopic19',
       'other_ftopic0', 'other_ftopic1', 'other_ftopic2', 'other_ftopic3',
       'other_ftopic4', 'other_ftopic5', 'other_ftopic6', 'other_ftopic7',
       'other_ftopic8', 'other_ftopic9', 'other_ftopic10', 'other_ftopic11',
       'other_ftopic12', 'other_ftopic13', 'other_ftopic14', 'other_ftopic15',
       'other_ftopic16', 'other_ftopic17', 'other_ftopic18', 'other_ftopic19',
       '#editors', 'ed_dialog_act_pred', 'ed_pred', 'all_dialog_act_pred',
       'all_pred'],
      dtype='object')

In [24]:
# Check for Keith's keys
current = old_allfeats
current_keys = set(zip(current['article'], current['thread'], current['editor']))
print(len(keith_keys.intersection(current_keys)))

52622


In [25]:
fold_df = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_folds.csv')
fold_df.columns

Index(['article', 'thread', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score', 'fold'],
      dtype='object')

In [26]:
ftopics_df = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_ftopics.csv')
ftopics_df.columns

Index(['article', 'thread', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score', 'ftopic0', 'ftopic1',
       'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6', 'ftopic7',
       'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12', 'ftopic13',
       'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18', 'ftopic19',
       'other_ftopic0', 'other_ftopic1', 'other_ftopic2', 'other_ftopic3',
       'other_ftopic4', 'other_ftopic5', 'other_ftopic6', 'other_ftopic7',
       'other_ftopic8', 'other_ftopic9', 'other_ftopic10', 'other_ftopic11',
       'other_ftopic12', 'other_ftopic13', 'other_ftopic14', 'other_ftopic15',
       'other_ftopic16', 'other_ftopic17', 'other_ftopic18', 'other_ftopic19'],
      dtype='object')

In [27]:
merged = pd.merge(fold_df, ftopics_df)
merged.columns

Index(['article', 'thread', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score', 'fold', 'ftopic0',
       'ftopic1', 'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6',
       'ftopic7', 'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12',
       'ftopic13', 'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18',
       'ftopic19', 'other_ftopic0', 'other_ftopic1', 'other_ftopic2',
       'other_ftopic3', 'other_ftopic4', 'other_ftopic5', 'other_ftopic6',
       'other_ftopic7', 'other_ftopic8', 'other_ftopic9', 'other_ftopic10',
       'other_ftopic11', 'other_ftopic12', 'other_ftopic13', 'other_ftopic14',
       'other_ftopic15', 'other_ftopic16', 'other_ftopic17', 'other_ftopic18',
       'other_ftopic19'],
      dtype='object')

In [28]:
# Check for Keith's keys
current = merged
current_keys = set(zip(current['article'], current['thread'], current['editor']))
print(len(keith_keys.intersection(current_keys)))

53175


In [30]:
merged.to_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_allfeats.csv', index=False)

### Add in #editors

In [31]:
feats=merged

In [32]:
# Group test folds by conversation size
gped = feats.groupby(['article', 'thread'])
convo_size = pd.DataFrame([(row[0][0], row[0][1], row[1]) for row in gped.size().iteritems()], columns=['article', 'thread', '#editors'])
convo_size

,article,thread,#editors
0,"""Tiger"" William Dunlop",Current image,3
1,"""Tiger"" William Dunlop",Requested move,12
2,"""V"" Is for Vagina","Why is the ""i"" capitalized?",8
3,&,"""O"" as a word",3
4,&,And per se and as a letter,2
5,&,Evolution,4
6,&,Handwriting,1
7,&,"Hebrew crowns, Egyptian cobras, Augustine, Per...",1
8,&,How common the misconception,2
9,&,Image,2


In [33]:
merged = pd.merge(feats, convo_size)
merged.columns

Index(['article', 'thread', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score', 'fold', 'ftopic0',
       'ftopic1', 'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6',
       'ftopic7', 'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12',
       'ftopic13', 'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18',
       'ftopic19', 'other_ftopic0', 'other_ftopic1', 'other_ftopic2',
       'other_ftopic3', 'other_ftopic4', 'other_ftopic5', 'other_ftopic6',
       'other_ftopic7', 'other_ftopic8', 'other_ftopic9', 'other_ftopic10',
       'other_ftopic11', 'other_ftopic12', 'other_ftopic13', 'other_ftopic14',
       'other_ftopic15', 'other_ftopic16', 'other_ftopic17', 'other_ftopic18',
       'other_ftopic19', '#editors'],
      dtype='object')

In [36]:
print(min(merged['#editors']))
print(max(merged['#editors']))

1
20


In [37]:
# Remove singletons
merged = merged[merged['#editors']>1]

In [46]:
# Check for Keith's keys
current = restricted
current_keys = set(zip(current['article'], current['thread'], current['editor']))
print(len(keith_keys.intersection(current_keys)))

53175


In [39]:
len(merged)

56634

In [40]:
nodups = merged.drop_duplicates(keep='first')
len(nodups)

56601

In [41]:
len(set(zip(merged['article'], merged['thread'], merged['editor'])))

56601

In [42]:
# Restrict to just Keith's keys
restricted = pd.merge(keith_datapts, nodups)
restricted.columns

Index(['article', 'thread', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score', 'fold', 'ftopic0',
       'ftopic1', 'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6',
       'ftopic7', 'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12',
       'ftopic13', 'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18',
       'ftopic19', 'other_ftopic0', 'other_ftopic1', 'other_ftopic2',
       'other_ftopic3', 'other_ftopic4', 'other_ftopic5', 'other_ftopic6',
       'other_ftopic7', 'other_ftopic8', 'other_ftopic9', 'other_ftopic10',
       'other_ftopic11', 'other_ftopic12', 'other_ftopic13', 'other_ftopic14',
       'other_ftopic15', 'other_ftopic16', 'other_ftopic17', 'other_ftopic18',
       'other_ftopic19', '#editors'],
      dtype='object')

In [43]:
len(restricted)

53175

In [44]:
restricted.drop_duplicates(inplace=True, keep='first')
len(restricted)

53175

In [45]:
restricted.to_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_allfeats.csv', index=False)

### Get folds

In [3]:
fold_info = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/fold_info.tsv', sep='\t', header=None, names=['article', 'fold'])
fold_info

,article,fold
0,Ocean Park Cable Car,0
1,"Ipswich, Queensland",1
2,The Phantom Limbs,5
3,List of FLCL characters,8
4,Karkadann,3
5,List of Red Garden characters,4
6,List of religious leaders in 1847,5
7,List of whisky brands,7
8,John the Fearless,6
9,Grow a Pear,0


In [6]:
keith_datapts = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/instances.tsv', sep='\t', header=None, names=['article', 'thread', 'editor'])
keith_keys = set(zip(keith_datapts['article'], keith_datapts['thread'], keith_datapts['editor']))
len(keith_keys)

53175

In [10]:
keith_articles = set(keith_datapts['article'].unique())
keith_articles

{'Larry Hama',
 'Euro gold and silver commemorative coins (Finland)',
 'Experimental aircraft',
 'Willamette Week',
 'History of Xcode',
 'Sundiata Acoli',
 'Adverb',
 'Mount Hebron',
 "Stalin's Missed Chance",
 'New Weird',
 'Wilfrid',
 'Common Desktop Environment',
 'Camp Nelson National Cemetery',
 'Disinfectant',
 'Linda Flynn',
 'Keggy the Keg',
 'Sealed lead acid cell',
 '5555',
 "Residents' association",
 'Remand (court procedure)',
 'Comparison of online dating websites',
 'List of the most spoken native languages',
 'Pope John Paul the Great',
 'Pain perdu',
 "St. Kevin's Cell",
 'God as the Devil',
 'Fraudulent investment scheme',
 'Sarnia Photovoltaic Power Plant',
 'Kapala',
 'Stream processing',
 'Gianna Beretta Molla',
 "Frank Freeman's Barber Shop",
 'Benzylpiperazine',
 'Ramayana',
 'Rotini',
 'Palmaria palmata',
 'Value (computer science)',
 'Orochi (The King of Fighters)',
 'Warning beacons of Gondor',
 'Chromatic note',
 'Temple Rodef Shalom (Falls Church, Virginia)'

In [13]:
# Check to make sure Keith's articles still in there
len(keith_articles - keith_articles.intersection(set(fold_info['article'])))

0

In [14]:
# Merge with edit scores
scores = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores.csv')
scores.columns

Index(['article', 'thread_title', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score'],
      dtype='object')

In [15]:
merged = pd.merge(scores, fold_info)
print(len(merged))
merged.columns

64957


Index(['article', 'thread_title', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score', 'fold'],
      dtype='object')

In [18]:
# Check for Keith's keys
current = merged
current_keys = set(zip(current['article'], current['thread_title'], current['editor']))
print(len(keith_keys.intersection(current_keys)))

53175


In [19]:
merged.rename(columns={'thread_title': 'thread'}, inplace=True)
merged.columns

Index(['article', 'thread', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score', 'fold'],
      dtype='object')

In [20]:
merged.to_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_folds.csv', index=False)

### Get ftopics

#### Editor DAs

In [21]:
# Load sentence-level DAs
sent_das = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/HSTTM8W-talk0-S5-FT20-BT100-FA0.1-B0.001-FG1.0-K0.1-E0.75-N0.75-I10000-InstSentAssign.csv')
sent_das.columns

Index(['SeqId', 'InstNo', 'User', 'Sentence', 'TaggedText', 'State', 'FTopic'], dtype='object')

In [24]:
# Group things by seqid
grouped = sent_das.groupby(['SeqId', 'User'])

ftopic_counts = {}

for (thread, user), inds in tqdm(grouped.groups.items()):
    ftopics = Counter(sent_das.iloc[inds]['FTopic'].tolist())
    ftopic_counts[(thread, user)] = ftopics
    
thread_user_topics = pd.DataFrame(ftopic_counts).transpose()
thread_user_topics.fillna(0.0, inplace=True)
thread_user_topics = thread_user_topics[[float(x) for x in range(20)]]
thread_user_topics['editor'] = [item[1] for item in thread_user_topics.index]
thread_user_topics['article_name'] = [item[0].split('#', 1)[0] for item in thread_user_topics.index]
thread_user_topics['thread'] = [item[0].split('#', 1)[1] for item in thread_user_topics.index]
thread_user_topics.rename(columns={n: 'ftopic{:.0f}'.format(n) for n in range(20)}, inplace=True)
thread_user_topics

cols = thread_user_topics.columns.tolist()
new_cols = cols[-2:] + cols[-3:-2] + cols[:-3]
thread_user_topics

,ftopic0,ftopic1,ftopic2,ftopic3,ftopic4,ftopic5,ftopic6,ftopic7,ftopic8,ftopic9,...,ftopic13,ftopic14,ftopic15,ftopic16,ftopic17,ftopic18,ftopic19,editor,article_name,thread
"(""Tiger"" William Dunlop#Current image, Deconstructhis)",2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,2.0,0.0,Deconstructhis,"""Tiger"" William Dunlop",Current image
"(""Tiger"" William Dunlop#Current image, Ukexpat)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Ukexpat,"""Tiger"" William Dunlop",Current image
"(""Tiger"" William Dunlop#Current image, Skinsmoke)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Skinsmoke,"""Tiger"" William Dunlop",Current image
"(""Tiger"" William Dunlop#Current image, Canglesea)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,Canglesea,"""Tiger"" William Dunlop",Current image
"(""Tiger"" William Dunlop#Requested move, Billinghurst)",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,2.0,0.0,0.0,0.0,0.0,Billinghurst,"""Tiger"" William Dunlop",Requested move
"(""Tiger"" William Dunlop#Requested move, CJ3370)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,CJ3370,"""Tiger"" William Dunlop",Requested move
"(""Tiger"" William Dunlop#Requested move, Skinsmoke)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,Skinsmoke,"""Tiger"" William Dunlop",Requested move
"(""Tiger"" William Dunlop#Requested move, Deconstructhis)",2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Deconstructhis,"""Tiger"" William Dunlop",Requested move
"(""V"" Is for Vagina#Why is the ""i"" capitalized?, Jennavecia)",0.0,0.0,1.0,3.0,2.0,0.0,8.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,1.0,Jennavecia,"""V"" Is for Vagina","Why is the ""i"" capitalized?"
"(""V"" Is for Vagina#Why is the ""i"" capitalized?, Skomorokh)",0.0,0.0,0.0,0.0,0.0,0.0,10.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2.0,0.0,Skomorokh,"""V"" Is for Vagina","Why is the ""i"" capitalized?"


In [25]:
thread_user_topics = thread_user_topics[new_cols]
thread_user_topics

,article_name,thread,editor,ftopic0,ftopic1,ftopic2,ftopic3,ftopic4,ftopic5,ftopic6,...,ftopic10,ftopic11,ftopic12,ftopic13,ftopic14,ftopic15,ftopic16,ftopic17,ftopic18,ftopic19
"(""Tiger"" William Dunlop#Current image, Deconstructhis)","""Tiger"" William Dunlop",Current image,Deconstructhis,2.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0
"(""Tiger"" William Dunlop#Current image, Ukexpat)","""Tiger"" William Dunlop",Current image,Ukexpat,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
"(""Tiger"" William Dunlop#Current image, Skinsmoke)","""Tiger"" William Dunlop",Current image,Skinsmoke,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
"(""Tiger"" William Dunlop#Current image, Canglesea)","""Tiger"" William Dunlop",Current image,Canglesea,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
"(""Tiger"" William Dunlop#Requested move, Billinghurst)","""Tiger"" William Dunlop",Requested move,Billinghurst,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0
"(""Tiger"" William Dunlop#Requested move, CJ3370)","""Tiger"" William Dunlop",Requested move,CJ3370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
"(""Tiger"" William Dunlop#Requested move, Skinsmoke)","""Tiger"" William Dunlop",Requested move,Skinsmoke,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
"(""Tiger"" William Dunlop#Requested move, Deconstructhis)","""Tiger"" William Dunlop",Requested move,Deconstructhis,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
"(""V"" Is for Vagina#Why is the ""i"" capitalized?, Jennavecia)","""V"" Is for Vagina","Why is the ""i"" capitalized?",Jennavecia,0.0,0.0,1.0,3.0,2.0,0.0,8.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0
"(""V"" Is for Vagina#Why is the ""i"" capitalized?, Skomorokh)","""V"" Is for Vagina","Why is the ""i"" capitalized?",Skomorokh,0.0,0.0,0.0,0.0,0.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0


In [26]:
keith_keys = set(zip(keith_datapts['article'], keith_datapts['thread'], keith_datapts['editor']))
len(keith_keys)

53175

In [29]:
ftopic_keys = set(zip(thread_user_topics['article_name'], thread_user_topics['thread'], thread_user_topics['editor']))
print(len(keith_keys.intersection(ftopic_keys)))

53175


In [30]:
thread_user_topics.to_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_dialog_acts.csv', index=False)

#### Other DAs

In [2]:
# Load sentence-level DAs
sent_das = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/HSTTM8W-talk0-S5-FT20-BT100-FA0.1-B0.001-FG1.0-K0.1-E0.75-N0.75-I10000-InstSentAssign.csv')
sent_das.columns

Index(['SeqId', 'InstNo', 'User', 'Sentence', 'TaggedText', 'State', 'FTopic'], dtype='object')

In [3]:
# Group things by seqid and user
user_grouped = sent_das.groupby(['SeqId', 'User'])
grouped = sent_das.groupby(['SeqId'])

In [8]:
# Build up other ftopic counts
ftopic_other_counts = {}

for (thread, user), inds in tqdm(user_grouped.groups.items()):
    thread_inds = grouped.groups[thread]
    other_inds = list(set(thread_inds) - set(inds))
    ftopics_other = Counter(sent_das.iloc[other_inds]['FTopic'].tolist())
    ftopic_other_counts[(thread,user)] = ftopics_other
    
thread_other_topics = pd.DataFrame(ftopic_other_counts).transpose()
thread_other_topics = thread_other_topics[[float(x) for x in range(20)]]
thread_other_topics.fillna(0.0, inplace=True)
thread_other_topics

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0
"(""Tiger"" William Dunlop#Current image, Deconstructhis)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0
"(""Tiger"" William Dunlop#Current image, Ukexpat)",2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,0.0
"(""Tiger"" William Dunlop#Current image, Skinsmoke)",2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,0.0
"(""Tiger"" William Dunlop#Current image, Canglesea)",2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,4.0,0.0
"(""Tiger"" William Dunlop#Requested move, Billinghurst)",2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0,0.0
"(""Tiger"" William Dunlop#Requested move, CJ3370)",3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,3.0,0.0
"(""Tiger"" William Dunlop#Requested move, Skinsmoke)",3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,3.0,0.0
"(""Tiger"" William Dunlop#Requested move, Deconstructhis)",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,2.0,0.0
"(""V"" Is for Vagina#Why is the ""i"" capitalized?, Jennavecia)",0.0,0.0,0.0,0.0,1.0,0.0,18.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0
"(""V"" Is for Vagina#Why is the ""i"" capitalized?, Skomorokh)",0.0,0.0,1.0,3.0,3.0,0.0,16.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,2.0


In [9]:
# Manipulate columns
thread_other_topics['editor'] = [item[1] for item in thread_other_topics.index]
thread_other_topics['article'] = [item[0].split('#', 1)[0] for item in thread_other_topics.index]
thread_other_topics['thread'] = [item[0].split('#', 1)[1] for item in thread_other_topics.index]
thread_other_topics

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,13.0,14.0,15.0,16.0,17.0,18.0,19.0,editor,article,thread
"(""Tiger"" William Dunlop#Current image, Deconstructhis)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,2.0,2.0,0.0,Deconstructhis,"""Tiger"" William Dunlop",Current image
"(""Tiger"" William Dunlop#Current image, Ukexpat)",2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,3.0,3.0,0.0,Ukexpat,"""Tiger"" William Dunlop",Current image
"(""Tiger"" William Dunlop#Current image, Skinsmoke)",2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,1.0,3.0,3.0,0.0,Skinsmoke,"""Tiger"" William Dunlop",Current image
"(""Tiger"" William Dunlop#Current image, Canglesea)",2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,1.0,1.0,4.0,0.0,Canglesea,"""Tiger"" William Dunlop",Current image
"(""Tiger"" William Dunlop#Requested move, Billinghurst)",2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,2.0,0.0,0.0,4.0,0.0,Billinghurst,"""Tiger"" William Dunlop",Requested move
"(""Tiger"" William Dunlop#Requested move, CJ3370)",3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,2.0,3.0,0.0,0.0,3.0,0.0,CJ3370,"""Tiger"" William Dunlop",Requested move
"(""Tiger"" William Dunlop#Requested move, Skinsmoke)",3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,2.0,3.0,0.0,0.0,3.0,0.0,Skinsmoke,"""Tiger"" William Dunlop",Requested move
"(""Tiger"" William Dunlop#Requested move, Deconstructhis)",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,2.0,4.0,0.0,0.0,2.0,0.0,Deconstructhis,"""Tiger"" William Dunlop",Requested move
"(""V"" Is for Vagina#Why is the ""i"" capitalized?, Jennavecia)",0.0,0.0,0.0,0.0,1.0,0.0,18.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,5.0,1.0,Jennavecia,"""V"" Is for Vagina","Why is the ""i"" capitalized?"
"(""V"" Is for Vagina#Why is the ""i"" capitalized?, Skomorokh)",0.0,0.0,1.0,3.0,3.0,0.0,16.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,2.0,Skomorokh,"""V"" Is for Vagina","Why is the ""i"" capitalized?"


In [11]:
keith_datapts = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/instances.tsv', sep='\t', header=None, names=['article', 'thread', 'editor'])
len(keith_datapts)

53175

In [12]:
keith_keys = set(zip(keith_datapts['article'], keith_datapts['thread'], keith_datapts['editor']))
len(keith_keys)

53175

In [19]:
ftopic_keys = set(zip(thread_other_topics['article'], thread_other_topics['thread'], thread_other_topics['editor']))

In [20]:
print(len(keith_keys.intersection(ftopic_keys)))

53175


#### Merge editor DAs and other DAs

In [31]:
print(thread_user_topics.columns)
print(thread_other_topics.columns)

Index(['article_name', 'thread', 'editor', 'ftopic0', 'ftopic1', 'ftopic2',
       'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6', 'ftopic7', 'ftopic8',
       'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12', 'ftopic13', 'ftopic14',
       'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18', 'ftopic19'],
      dtype='object')
Index([      0.0,       1.0,       2.0,       3.0,       4.0,       5.0,
             6.0,       7.0,       8.0,       9.0,      10.0,      11.0,
            12.0,      13.0,      14.0,      15.0,      16.0,      17.0,
            18.0,      19.0,  'editor', 'article',  'thread'],
      dtype='object')


In [32]:
thread_other_topics.rename(columns={x: 'other_ftopic{:d}'.format(int(x)) for x in range(20)}, inplace=True)
thread_other_topics.columns

Index(['other_ftopic0', 'other_ftopic1', 'other_ftopic2', 'other_ftopic3',
       'other_ftopic4', 'other_ftopic5', 'other_ftopic6', 'other_ftopic7',
       'other_ftopic8', 'other_ftopic9', 'other_ftopic10', 'other_ftopic11',
       'other_ftopic12', 'other_ftopic13', 'other_ftopic14', 'other_ftopic15',
       'other_ftopic16', 'other_ftopic17', 'other_ftopic18', 'other_ftopic19',
       'editor', 'article', 'thread'],
      dtype='object')

In [33]:
thread_user_topics.rename(columns={'article_name': 'article'}, inplace=True)
thread_user_topics.columns

Index(['article', 'thread', 'editor', 'ftopic0', 'ftopic1', 'ftopic2',
       'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6', 'ftopic7', 'ftopic8',
       'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12', 'ftopic13', 'ftopic14',
       'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18', 'ftopic19'],
      dtype='object')

In [34]:
merged = pd.merge(thread_user_topics, thread_other_topics)
merged.columns

Index(['article', 'thread', 'editor', 'ftopic0', 'ftopic1', 'ftopic2',
       'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6', 'ftopic7', 'ftopic8',
       'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12', 'ftopic13', 'ftopic14',
       'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18', 'ftopic19',
       'other_ftopic0', 'other_ftopic1', 'other_ftopic2', 'other_ftopic3',
       'other_ftopic4', 'other_ftopic5', 'other_ftopic6', 'other_ftopic7',
       'other_ftopic8', 'other_ftopic9', 'other_ftopic10', 'other_ftopic11',
       'other_ftopic12', 'other_ftopic13', 'other_ftopic14', 'other_ftopic15',
       'other_ftopic16', 'other_ftopic17', 'other_ftopic18', 'other_ftopic19'],
      dtype='object')

In [35]:
merged.to_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_dialog_acts.csv', index=False)

In [36]:
# Merge with edit scores
scores = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores.csv')
scores.columns

Index(['article', 'thread_title', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score'],
      dtype='object')

In [37]:
scores.rename(columns={'thread_title': 'thread'}, inplace=True)
scores.columns

Index(['article', 'thread', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score'],
      dtype='object')

In [38]:
merged_all = pd.merge(scores, merged)
print(merged_all.columns)
len(merged_all)

Index(['article', 'thread', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score', 'ftopic0', 'ftopic1',
       'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6', 'ftopic7',
       'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12', 'ftopic13',
       'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18', 'ftopic19',
       'other_ftopic0', 'other_ftopic1', 'other_ftopic2', 'other_ftopic3',
       'other_ftopic4', 'other_ftopic5', 'other_ftopic6', 'other_ftopic7',
       'other_ftopic8', 'other_ftopic9', 'other_ftopic10', 'other_ftopic11',
       'other_ftopic12', 'other_ftopic13', 'other_ftopic14', 'other_ftopic15',
       'other_ftopic16', 'other_ftopic17', 'other_ftopic18', 'other_ftopic19'],
      dtype='object')


69311

In [39]:
current = merged_all
current_keys = set(zip(current['article'], current['thread'], current['editor']))
print(len(keith_keys.intersection(current_keys)))

53175


In [40]:
current.to_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_ftopics.csv', index=False)

## Investigate missing data

In [10]:
keith_datapts = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/instances.tsv', sep='\t', header=None, names=['article', 'thread', 'editor'])
len(keith_datapts)

53175

In [5]:
feats = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_allfeats.csv')
len(feats)

56810

In [6]:
merged = pd.merge(feats, keith_datapts, how='right')
print(len(merged))
merged.columns

53961


Index(['fold', 'article', 'thread', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score', 'ftopic0', 'ftopic1',
       'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6', 'ftopic7',
       'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12', 'ftopic13',
       'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18', 'ftopic19',
       'other_ftopic0', 'other_ftopic1', 'other_ftopic2', 'other_ftopic3',
       'other_ftopic4', 'other_ftopic5', 'other_ftopic6', 'other_ftopic7',
       'other_ftopic8', 'other_ftopic9', 'other_ftopic10', 'other_ftopic11',
       'other_ftopic12', 'other_ftopic13', 'other_ftopic14', 'other_ftopic15',
       'other_ftopic16', 'other_ftopic17', 'other_ftopic18', 'other_ftopic19',
       '#editors', 'ed_dialog_act_pred', 'ed_pred', 'all_dialog_act_pred',
       'all_pred'],
      dtype='object')

In [7]:
merged = pd.merge(feats, keith_datapts, how='inner')
print(len(merged))
merged.columns

53408


Index(['fold', 'article', 'thread', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score', 'ftopic0', 'ftopic1',
       'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6', 'ftopic7',
       'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12', 'ftopic13',
       'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18', 'ftopic19',
       'other_ftopic0', 'other_ftopic1', 'other_ftopic2', 'other_ftopic3',
       'other_ftopic4', 'other_ftopic5', 'other_ftopic6', 'other_ftopic7',
       'other_ftopic8', 'other_ftopic9', 'other_ftopic10', 'other_ftopic11',
       'other_ftopic12', 'other_ftopic13', 'other_ftopic14', 'other_ftopic15',
       'other_ftopic16', 'other_ftopic17', 'other_ftopic18', 'other_ftopic19',
       '#editors', 'ed_dialog_act_pred', 'ed_pred', 'all_dialog_act_pred',
       'all_pred'],
      dtype='object')

In [8]:
merged_keys = set(zip(merged['article'], merged['thread'], merged['editor']))
len(merged_keys)

52622

In [9]:
keith_keys = set(zip(keith_datapts['article'], keith_datapts['thread'], keith_datapts['editor']))
len(keith_keys)

53175

In [10]:
feats_keys = set(zip(feats['article'], feats['thread'], feats['editor']))
len(feats_keys)

56021

In [11]:
# Find rows that Keith has that I don't
missing = keith_keys-merged_keys
print(len(missing))
missing

553


{('"Tiger" William Dunlop', 'Requested move', 'Skinsmoke'),
 ('2009 NBA Eastern Conference Playoffs',
  "4-way tie for the West's #2",
  'Howard the Duck'),
 ('2009 NBA Eastern Conference Playoffs',
  "4-way tie for the West's #2",
  'Martin tamb'),
 ('2009 NBA Eastern Conference Playoffs',
  "4-way tie for the West's #2",
  'Mild Bill Hiccup'),
 ('2009 NBA Eastern Conference Playoffs',
  "4-way tie for the West's #2",
  'MrArticleOne'),
 ('2010 WPA World Nine-ball Championship', 'Expansion', 'Howard the Duck'),
 ('240i', 'Virtual Console', 'Link83'),
 ('33550336 (number)',
  '"not distinct prime factors" vs not \'necessarily distinct prime factors for odd perf #s',
  'David Eppstein'),
 ('33550336 (number)',
  '"not distinct prime factors" vs not \'necessarily distinct prime factors for odd perf #s',
  'Pmanderson'),
 ('33550336 (number)',
  '"not distinct prime factors" vs not \'necessarily distinct prime factors for odd perf #s',
  'Richard L. Peterson'),
 ('826 Naval Air Squadron',

In [12]:
# Find missing rows in data
scores = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores.csv')
scores.columns

Index(['article', 'thread_title', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score'],
      dtype='object')

In [23]:
# Search for missing row
egs = random.sample(list(missing), 10)
egs

[('Oseltamivir', 'U.S. Government policy', 'Literaturegeek'),
 ('Juni and Juli',
  'More effort finding sources for individuals',
  'Lucia Black'),
 ('Toonami (Rebirth)-Adult Swim', 'Multiple "One Piece" entries', 'Takuy'),
 ('Indian Springs Army Airfield', 'Second source yet?', 'Hcobb'),
 ('Termial function', 'An elegant property#2 and #3', 'David Eppstein'),
 ('Chris (The King of Fighters)', 'Adelheid', 'Sake neko'),
 ('Vilnius',
  '"German Occupation" versus "In Poland" and "In Russia"',
  'Renata3'),
 ('Batman (TV series)', 'Batman jokes and theme', 'ChrisGriswold'),
 ('"Tiger" William Dunlop', 'Requested move', 'Skinsmoke'),
 ('Smells like teens spirit',
  'Smells Like Teen Spirit:  #1 on Single Sales Chart',
  'Allaplgies')]

In [16]:
scores_keys = set(zip(scores['article'], scores['thread_title'], scores['editor']))
len(scores_keys)

69319

In [24]:
all(eg in scores_keys for eg in egs)

True

In [26]:
pd.set_option('display.max_colwidth', 999)

In [27]:
mask = list(map(lambda tup: tup in egs, zip(scores['article'], scores['thread_title'], scores['editor'])))
scores[mask]

,article,thread_title,editor,editor_talk,other_talk,#editor_turns,#other_turns,editor_score
7,"""Tiger"" William Dunlop",Requested move,Skinsmoke,"User:CJ3370/William_Tiger_Dunlop → William Tiger Dunlop Oppose This will result in two articles on the same person, one under William Tiger Dunlop and the other under William ""Tiger"" Dunlop. We do not improve articles by shunting the original off into another title. The way to deal with this is to merge the improvements into the existing article. (talk) 21:12, 14 July 2010 (UTC) Comment Sorry that you took offense, but it wasn't a ""royal"" we; it's meaning was that we, as Wikipedia editors or the Wikipedia community, don't do it that way. Now to add being patronising to my crimes: you've done an excellent job with the merge. Have you really only been on Wikipedia for a month? Don't worry about getting things wrong every so often. We (and I mean we) are all still learning, and there's not much you can do that can't be undone if a mistake is made. Enjoy your time on here, and if this article is anything to go by, may you be here a long time! (talk) 01:49, 15 July 2010 (UTC)","The following discussion is an archived discussion of a requested move. Please do not modify it. Subsequent comments should be made in a new section on the talk page. No further edits should be made to this section. The result of the move request was: move lapsed, article merge preferred billinghurst sDrewth 23:13, 21 July 2010 (UTC) William ""Tiger"" Dunlop → William ""Tiger"" Dunlop — Move to provide a more substancial bio of Tiger Dunlop. CJ_WeißSchäfer 16:47, 14 July 2010 (UTC) I'd like to make this contribution to Tiger Dunlop's bio.CJ_WeißSchäfer 16:47, 14 July 2010 (UTC) Not a problem to merge if that is the most appropriate way to accomplish a decent article on Tiger. I am new to wikiland so bear with me and try not to use the royal ""we"" with a little high handed; there is more neutral way to say that. —Preceding unsigned comment added by (talk • contribs) 22:38, 14 July 2010 (UTC) Excellent job! Dunlop has been deserving of a much more informative presentation of both his int...",3,5,0.456897
8,"""Tiger"" William Dunlop",Requested move,Skinsmoke,"User:CJ3370/William_Tiger_Dunlop → William Tiger Dunlop Oppose This will result in two articles on the same person, one under William Tiger Dunlop and the other under William ""Tiger"" Dunlop. We do not improve articles by shunting the original off into another title. The way to deal with this is to merge the improvements into the existing article. (talk) 21:12, 14 July 2010 (UTC) Comment Sorry that you took offense, but it wasn't a ""royal"" we; it's meaning was that we, as Wikipedia editors or the Wikipedia community, don't do it that way. Now to add being patronising to my crimes: you've done an excellent job with the merge. Have you really only been on Wikipedia for a month? Don't worry about getting things wrong every so often. We (and I mean we) are all still learning, and there's not much you can do that can't be undone if a mistake is made. Enjoy your time on here, and if this article is anything to go by, may you be here a long time! (talk) 01:49, 15 July 2010 (UTC)","The following discussion is an archived discussion of a requested move. Please do not modify it. Subsequent comments should be made in a new section on the talk page. No further edits should be made to this section. The result of the move request was: move lapsed, article merge preferred billinghurst sDrewth 23:13, 21 July 2010 (UTC) William ""Tiger"" Dunlop → William ""Tiger"" Dunlop — Move to provide a more substancial bio of Tiger Dunlop. CJ_WeißSchäfer 16:47, 14 July 2010 (UTC) I'd like to make this contribution to Tiger Dunlop's bio.CJ_WeißSchäfer 16:47, 14 July 2010 (UTC) Not a problem to merge if that is the most appropriate way to accomplish a decent article on Tiger. I am new to wikiland so bear with me and try not to use the royal ""we"" with a little high handed; there is m

## Check in ftopics

In [29]:
df = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_ftopics.csv')
df.columns

Index(['article', 'thread_title', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score', 'ftopic0', 'ftopic1',
       'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6', 'ftopic7',
       'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12', 'ftopic13',
       'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18', 'ftopic19',
       'other_ftopic0', 'other_ftopic1', 'other_ftopic2', 'other_ftopic3',
       'other_ftopic4', 'other_ftopic5', 'other_ftopic6', 'other_ftopic7',
       'other_ftopic8', 'other_ftopic9', 'other_ftopic10', 'other_ftopic11',
       'other_ftopic12', 'other_ftopic13', 'other_ftopic14', 'other_ftopic15',
       'other_ftopic16', 'other_ftopic17', 'other_ftopic18', 'other_ftopic19'],
      dtype='object')

In [37]:
fields = ['article', 'thread_title', 'editor']
keys = set(zip(df[fields[0]], df[fields[1]], df[fields[2]]))
print(len(keys))
print(all(eg in keys for eg in egs))
print(any(eg in keys for eg in egs))
print()

for eg in egs:
    print(eg in keys, end=':\t')
    print(eg)

69203
False
True

True:	('Oseltamivir', 'U.S. Government policy', 'Literaturegeek')
True:	('Juni and Juli', 'More effort finding sources for individuals', 'Lucia Black')
True:	('Toonami (Rebirth)-Adult Swim', 'Multiple "One Piece" entries', 'Takuy')
True:	('Indian Springs Army Airfield', 'Second source yet?', 'Hcobb')
False:	('Termial function', 'An elegant property#2 and #3', 'David Eppstein')
True:	('Chris (The King of Fighters)', 'Adelheid', 'Sake neko')
True:	('Vilnius', '"German Occupation" versus "In Poland" and "In Russia"', 'Renata3')
True:	('Batman (TV series)', 'Batman jokes and theme', 'ChrisGriswold')
True:	('"Tiger" William Dunlop', 'Requested move', 'Skinsmoke')
False:	('Smells like teens spirit', 'Smells Like Teen Spirit:  #1 on Single Sales Chart', 'Allaplgies')


In [34]:
mask = list(map(lambda tup: tup in egs, zip(df[fields[0]], df[fields[1]], df[fields[2]])))
selected = df[mask]
print(len(selected))
selected

9


,article,thread_title,editor,editor_talk,other_talk,#editor_turns,#other_turns,editor_score,ftopic0,ftopic1,...,other_ftopic10,other_ftopic11,other_ftopic12,other_ftopic13,other_ftopic14,other_ftopic15,other_ftopic16,other_ftopic17,other_ftopic18,other_ftopic19
7,"""Tiger"" William Dunlop",Requested move,Skinsmoke,"User:CJ3370/William_Tiger_Dunlop → William Tiger Dunlop Oppose This will result in two articles on the same person, one under William Tiger Dunlop and the other under William ""Tiger"" Dunlop. We do not improve articles by shunting the original off into another title. The way to deal with this is to merge the improvements into the existing article. (talk) 21:12, 14 July 2010 (UTC) Comment Sorry that you took offense, but it wasn't a ""royal"" we; it's meaning was that we, as Wikipedia editors or the Wikipedia community, don't do it that way. Now to add being patronising to my crimes: you've done an excellent job with the merge. Have you really only been on Wikipedia for a month? Don't worry about getting things wrong every so often. We (and I mean we) are all still learning, and there's not much you can do that can't be undone if a mistake is made. Enjoy your time on here, and if this article is anything to go by, may you be here a long time! (talk) 01:49, 15 July 2010 (UTC)","The following discussion is an archived discussion of a requested move. Please do not modify it. Subsequent comments should be made in a new section on the talk page. No further edits should be made to this section. The result of the move request was: move lapsed, article merge preferred billinghurst sDrewth 23:13, 21 July 2010 (UTC) William ""Tiger"" Dunlop → William ""Tiger"" Dunlop — Move to provide a more substancial bio of Tiger Dunlop. CJ_WeißSchäfer 16:47, 14 July 2010 (UTC) I'd like to make this contribution to Tiger Dunlop's bio.CJ_WeißSchäfer 16:47, 14 July 2010 (UTC) Not a problem to merge if that is the most appropriate way to accomplish a decent article on Tiger. I am new to wikiland so bear with me and try not to use the royal ""we"" with a little high handed; there is more neutral way to say that. —Preceding unsigned comment added by (talk • contribs) 22:38, 14 July 2010 (UTC) Excellent job! Dunlop has been deserving of a much more informative presentation of both his int...",3,5,0.456897,0.0,0.0,...,1.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,3.0,0.0
8,"""Tiger"" William Dunlop",Requested move,Skinsmoke,"User:CJ3370/William_Tiger_Dunlop → William Tiger Dunlop Oppose This will result in two articles on the same person, one under William Tiger Dunlop and the other under William ""Tiger"" Dunlop. We do not improve articles by shunting the original off into another title. The way to deal with this is to merge the improvements into the existing article. (talk) 21:12, 14 July 2010 (UTC) Comment Sorry that you took offense, but it wasn't a ""royal"" we; it's meaning was that we, as Wikipedia editors or the Wikipedia community, don't do it that way. Now to add being patronising to my crimes: you've done an excellent job with the merge. Have you really only been on Wikipedia for a month? Don't worry about getting things wrong every so often. We (and I mean we) are all still learning, and there's not much you can do that can't be undone if a mistake is made. Enjoy your time on here, and if this article is anything to go by, may you be here a long time! (talk) 01:49, 15 July 2010 (UTC)","The following discussion is an archived discussion of a requested move. Please do not modify it. Subsequent comments should be made in a new section on the talk page. No further edits should be made to this section. The result of the move request was: move lapsed, article merge preferred billinghurst sDrewth 23:13, 21 July 2010 (UTC) William ""Tiger"" Dunlop → William ""Tiger"" Dunlop — Move to provide a more substancial bio of Tiger Dunlop. CJ_WeißSchäfer 16:47, 14 July 2010 (UTC) I'd like to make this contribution to Tiger Dunlop's bio.CJ_WeißSchäfer 16:47, 14 July

## Check in dialogue acts

In [51]:
df = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_dialog_acts.csv')
df.columns

Index(['article_name', 'thread', 'editor', 'ftopic0', 'ftopic1', 'ftopic2',
       'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6', 'ftopic7', 'ftopic8',
       'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12', 'ftopic13', 'ftopic14',
       'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18', 'ftopic19'],
      dtype='object')

In [54]:
fields = ['article_name', 'thread', 'editor']
keys = set(zip(df[fields[0]], df[fields[1]], df[fields[2]]))
print(len(keys))
print(all(eg in keys for eg in egs))
print(any(eg in keys for eg in egs))
print()

for eg in egs:
    print(eg in keys, end=':\t')
    print(eg)

105641
False
True

True:	('Oseltamivir', 'U.S. Government policy', 'Literaturegeek')
True:	('Juni and Juli', 'More effort finding sources for individuals', 'Lucia Black')
True:	('Toonami (Rebirth)-Adult Swim', 'Multiple "One Piece" entries', 'Takuy')
True:	('Indian Springs Army Airfield', 'Second source yet?', 'Hcobb')
False:	('Termial function', 'An elegant property#2 and #3', 'David Eppstein')
True:	('Chris (The King of Fighters)', 'Adelheid', 'Sake neko')
True:	('Vilnius', '"German Occupation" versus "In Poland" and "In Russia"', 'Renata3')
True:	('Batman (TV series)', 'Batman jokes and theme', 'ChrisGriswold')
True:	('"Tiger" William Dunlop', 'Requested move', 'Skinsmoke')
False:	('Smells like teens spirit', 'Smells Like Teen Spirit:  #1 on Single Sales Chart', 'Allaplgies')


## Check in folds

In [38]:
df = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_folds.csv')
df.columns

Index(['article', 'thread_title', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score', 'article_name',
       'thread', 'ftopic0', 'ftopic1', 'ftopic2', 'ftopic3', 'ftopic4',
       'ftopic5', 'ftopic6', 'ftopic7', 'ftopic8', 'ftopic9', 'ftopic10',
       'ftopic11', 'ftopic12', 'ftopic13', 'ftopic14', 'ftopic15', 'ftopic16',
       'ftopic17', 'ftopic18', 'ftopic19', 'fold'],
      dtype='object')

In [39]:
fields = ['article', 'thread_title', 'editor']
keys = set(zip(df[fields[0]], df[fields[1]], df[fields[2]]))
print(len(keys))
print(all(eg in keys for eg in egs))
print(any(eg in keys for eg in egs))
print()

for eg in egs:
    print(eg in keys, end=':\t')
    print(eg)

62584
False
True

True:	('Oseltamivir', 'U.S. Government policy', 'Literaturegeek')
True:	('Juni and Juli', 'More effort finding sources for individuals', 'Lucia Black')
True:	('Toonami (Rebirth)-Adult Swim', 'Multiple "One Piece" entries', 'Takuy')
True:	('Indian Springs Army Airfield', 'Second source yet?', 'Hcobb')
False:	('Termial function', 'An elegant property#2 and #3', 'David Eppstein')
True:	('Chris (The King of Fighters)', 'Adelheid', 'Sake neko')
True:	('Vilnius', '"German Occupation" versus "In Poland" and "In Russia"', 'Renata3')
True:	('Batman (TV series)', 'Batman jokes and theme', 'ChrisGriswold')
True:	('"Tiger" William Dunlop', 'Requested move', 'Skinsmoke')
False:	('Smells like teens spirit', 'Smells Like Teen Spirit:  #1 on Single Sales Chart', 'Allaplgies')


In [40]:
mask = list(map(lambda tup: tup in egs, zip(df[fields[0]], df[fields[1]], df[fields[2]])))
selected = df[mask]
print(len(selected))
selected

9


,article,thread_title,editor,editor_talk,other_talk,#editor_turns,#other_turns,editor_score,article_name,thread,...,ftopic11,ftopic12,ftopic13,ftopic14,ftopic15,ftopic16,ftopic17,ftopic18,ftopic19,fold
7,"""Tiger"" William Dunlop",Requested move,Skinsmoke,"User:CJ3370/William_Tiger_Dunlop → William Tiger Dunlop Oppose This will result in two articles on the same person, one under William Tiger Dunlop and the other under William ""Tiger"" Dunlop. We do not improve articles by shunting the original off into another title. The way to deal with this is to merge the improvements into the existing article. (talk) 21:12, 14 July 2010 (UTC) Comment Sorry that you took offense, but it wasn't a ""royal"" we; it's meaning was that we, as Wikipedia editors or the Wikipedia community, don't do it that way. Now to add being patronising to my crimes: you've done an excellent job with the merge. Have you really only been on Wikipedia for a month? Don't worry about getting things wrong every so often. We (and I mean we) are all still learning, and there's not much you can do that can't be undone if a mistake is made. Enjoy your time on here, and if this article is anything to go by, may you be here a long time! (talk) 01:49, 15 July 2010 (UTC)","The following discussion is an archived discussion of a requested move. Please do not modify it. Subsequent comments should be made in a new section on the talk page. No further edits should be made to this section. The result of the move request was: move lapsed, article merge preferred billinghurst sDrewth 23:13, 21 July 2010 (UTC) William ""Tiger"" Dunlop → William ""Tiger"" Dunlop — Move to provide a more substancial bio of Tiger Dunlop. CJ_WeißSchäfer 16:47, 14 July 2010 (UTC) I'd like to make this contribution to Tiger Dunlop's bio.CJ_WeißSchäfer 16:47, 14 July 2010 (UTC) Not a problem to merge if that is the most appropriate way to accomplish a decent article on Tiger. I am new to wikiland so bear with me and try not to use the royal ""we"" with a little high handed; there is more neutral way to say that. —Preceding unsigned comment added by (talk • contribs) 22:38, 14 July 2010 (UTC) Excellent job! Dunlop has been deserving of a much more informative presentation of both his int...",3,5,0.456897,"""Tiger"" William Dunlop",Requested move,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,7.0
8,"""Tiger"" William Dunlop",Requested move,Skinsmoke,"User:CJ3370/William_Tiger_Dunlop → William Tiger Dunlop Oppose This will result in two articles on the same person, one under William Tiger Dunlop and the other under William ""Tiger"" Dunlop. We do not improve articles by shunting the original off into another title. The way to deal with this is to merge the improvements into the existing article. (talk) 21:12, 14 July 2010 (UTC) Comment Sorry that you took offense, but it wasn't a ""royal"" we; it's meaning was that we, as Wikipedia editors or the Wikipedia community, don't do it that way. Now to add being patronising to my crimes: you've done an excellent job with the merge. Have you really only been on Wikipedia for a month? Don't worry about getting things wrong every so often. We (and I mean we) are all still learning, and there's not much you can do that can't be undone if a mistake is made. Enjoy your time on here, and if this article is anything to go by, may you be here a long time! (talk) 01:49, 15 July 2010 (UTC)","The following discussion is an archived discussion of a requested move. Please do not modify it. Subsequent comments should be made in a new section on the talk page. No further edits should be made to this section. The result of the move request was: move lapsed, article merge preferred billinghurst sDrewth 23:13, 21 July 2010 (UTC) William ""Tiger"" Dunlop → William ""Tiger"" Dunlop — Move to provide a more substancial bio of Tiger Dunlop. CJ_WeißSchäfer 16:47, 14 July 2010 (UTC) I'd like to make this contribution to Tiger Dunlop's bio.CJ_WeißSchäfer 16:47, 14 July 2010 (UTC) Not a problem 

## Check if singletons

In [45]:
eg_threads = [(tup[0],tup[1]) for tup in egs]
eg_threads

[('Oseltamivir', 'U.S. Government policy'),
 ('Juni and Juli', 'More effort finding sources for individuals'),
 ('Toonami (Rebirth)-Adult Swim', 'Multiple "One Piece" entries'),
 ('Indian Springs Army Airfield', 'Second source yet?'),
 ('Termial function', 'An elegant property#2 and #3'),
 ('Chris (The King of Fighters)', 'Adelheid'),
 ('Vilnius', '"German Occupation" versus "In Poland" and "In Russia"'),
 ('Batman (TV series)', 'Batman jokes and theme'),
 ('"Tiger" William Dunlop', 'Requested move'),
 ('Smells like teens spirit',
  'Smells Like Teen Spirit:  #1 on Single Sales Chart')]

In [50]:
gped = scores.groupby(['article', 'thread_title'])
thread_sizes = gped.size()

for t in eg_threads:
    print(thread_sizes[t])

2
4
3
2
2
2
4
4
6
2


# Check dates of data

In [2]:
talk = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/talk_filtered_2.csv', parse_dates=['timestamp'])
max(talk['timestamp'])

Timestamp('2014-07-07 19:13:24')

# Add conversation size to features

In [44]:
# Load features
feats = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_allfeats.csv')
feats

,fold,article,thread,editor,editor_talk,other_talk,#editor_turns,#other_turns,editor_score,ftopic0,...,ftopic10,ftopic11,ftopic12,ftopic13,ftopic14,ftopic15,ftopic16,ftopic17,ftopic18,ftopic19
0,7,"""Tiger"" William Dunlop",Current image,Deconstructhis,I've noticed that the current image that's acc...,I removed the deletion tag because the image i...,2,3,0.033058,2.0,...,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0
1,7,"""Tiger"" William Dunlop",Current image,Skinsmoke,::Good move ukexpat. That was really helpful (...,I've noticed that the current image that's acc...,1,4,0.482759,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,7,"""Tiger"" William Dunlop",Current image,Ukexpat,I removed the deletion tag because the image i...,I've noticed that the current image that's acc...,1,4,0.392157,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,7,"""Tiger"" William Dunlop",Requested move,CJ3370,"William ""Tiger"" Dunlop → William ""Tiger"" Dunlo...",The following discussion is an archived discus...,2,6,0.826415,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,7,"""Tiger"" William Dunlop",Requested move,CJ3370,"William ""Tiger"" Dunlop → William ""Tiger"" Dunlo...",The following discussion is an archived discus...,2,6,0.826415,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
5,7,"""Tiger"" William Dunlop",Requested move,Deconstructhis,Not a problem to merge if that is the most app...,The following discussion is an archived discus...,1,7,0.033058,2.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
6,7,"""Tiger"" William Dunlop",Requested move,Deconstructhis,Not a problem to merge if that is the most app...,The following discussion is an archived discus...,1,7,0.033058,2.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
7,7,"""Tiger"" William Dunlop",Requested move,Skinsmoke,User:CJ3370/William_Tiger_Dunlop → William Tig...,The following discussion is an archived discus...,3,5,0.456897,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
8,7,"""Tiger"" William Dunlop",Requested move,Skinsmoke,User:CJ3370/William_Tiger_Dunlop → William Tig...,The following discussion is an archived discus...,3,5,0.456897,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
9,8,"""V"" Is for Vagina","Why is the ""i"" capitalized?",Gunmetal Angel,"""I"" is supposed to be capatalized in ""is"" in t...",It doesn't make sense and it doesn't represent...,1,9,1.000000,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [46]:
# Group test folds by conversation size
gped = feats.groupby(['article', 'thread'])
gped.size()

article                        thread                                                                                                                         
"Tiger" William Dunlop         Current image                                                                                                                      3
                               Requested move                                                                                                                     6
"V" Is for Vagina              Why is the "i" capitalized?                                                                                                        4
&                              "O" as a word                                                                                                                      3
                               And per se and as a letter                                                                                                         2
                     

In [59]:
convo_size = pd.DataFrame([(row[0][0], row[0][1], row[1]) for row in gped.size().iteritems()], columns=['article', 'thread', '#editors'])
convo_size

,article,thread,#editors
0,"""Tiger"" William Dunlop",Current image,3
1,"""Tiger"" William Dunlop",Requested move,6
2,"""V"" Is for Vagina","Why is the ""i"" capitalized?",4
3,&,"""O"" as a word",3
4,&,And per se and as a letter,2
5,&,Evolution,2
6,&,How common the misconception,2
7,&,Image,2
8,&,My use,4
9,'Abd al-Ilah,Mixed Up?,2


In [61]:
merged = pd.merge(feats, convo_size, on=['article', 'thread'])
merged.columns

Index(['fold', 'article', 'thread', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score', 'ftopic0', 'ftopic1',
       'ftopic2', 'ftopic3', 'ftopic4', 'ftopic5', 'ftopic6', 'ftopic7',
       'ftopic8', 'ftopic9', 'ftopic10', 'ftopic11', 'ftopic12', 'ftopic13',
       'ftopic14', 'ftopic15', 'ftopic16', 'ftopic17', 'ftopic18', 'ftopic19',
       '#editors'],
      dtype='object')

In [62]:
len(merged)

56528

In [63]:
min(merged['#editors'])

2

In [64]:
max(merged['#editors'])

20

In [65]:
merged.to_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_allfeats.csv', index=False)

# Add new fold information to allfeats dataset

In [27]:
# Open tsv of fold info in Pandas
fold_info = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/fold_info2.tsv', sep='\t', header=None, names=['article', 'fold'])
fold_info

,article,fold
0,Ocean Park Cable Car,0
1,"Ipswich, Queensland",1
2,The Phantom Limbs,5
3,List of FLCL characters,8
4,Karkadann,3
5,List of Red Garden characters,4
6,List of religious leaders in 1847,5
7,List of whisky brands,7
8,John the Fearless,6
9,Grow a Pear,0


In [24]:
fold_artnames = set(fold_info[0])
len(fold_artnames)

7211

In [25]:
feats = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_ftopics.csv')
len(feats['article'].unique())

10374

In [33]:
# Merge in fold info
merged = pd.merge(feats, fold_info, on=['article'])
merged

,article,thread_title,editor,editor_talk,other_talk,#editor_turns,#other_turns,editor_score,article_name,thread,...,ftopic11,ftopic12,ftopic13,ftopic14,ftopic15,ftopic16,ftopic17,ftopic18,ftopic19,fold
0,"""Tiger"" William Dunlop",Current image,Deconstructhis,I've noticed that the current image that's acc...,I removed the deletion tag because the image i...,2,3,0.033058,"""Tiger"" William Dunlop",Current image,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,7
1,"""Tiger"" William Dunlop",Current image,Skinsmoke,::Good move ukexpat. That was really helpful (...,I've noticed that the current image that's acc...,1,4,0.482759,"""Tiger"" William Dunlop",Current image,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7
2,"""Tiger"" William Dunlop",Current image,Ukexpat,I removed the deletion tag because the image i...,I've noticed that the current image that's acc...,1,4,0.392157,"""Tiger"" William Dunlop",Current image,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7
3,"""Tiger"" William Dunlop",Requested move,CJ3370,"William ""Tiger"" Dunlop → William ""Tiger"" Dunlo...",The following discussion is an archived discus...,2,6,0.826415,"""Tiger"" William Dunlop",Requested move,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,7
4,"""Tiger"" William Dunlop",Requested move,CJ3370,"William ""Tiger"" Dunlop → William ""Tiger"" Dunlo...",The following discussion is an archived discus...,2,6,0.826415,"""Tiger"" William Dunlop",Requested move,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,7
5,"""Tiger"" William Dunlop",Requested move,Deconstructhis,Not a problem to merge if that is the most app...,The following discussion is an archived discus...,1,7,0.033058,"""Tiger"" William Dunlop",Requested move,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,7
6,"""Tiger"" William Dunlop",Requested move,Deconstructhis,Not a problem to merge if that is the most app...,The following discussion is an archived discus...,1,7,0.033058,"""Tiger"" William Dunlop",Requested move,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,7
7,"""Tiger"" William Dunlop",Requested move,Skinsmoke,User:CJ3370/William_Tiger_Dunlop → William Tig...,The following discussion is an archived discus...,3,5,0.456897,"""Tiger"" William Dunlop",Requested move,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,7
8,"""Tiger"" William Dunlop",Requested move,Skinsmoke,User:CJ3370/William_Tiger_Dunlop → William Tig...,The following discussion is an archived discus...,3,5,0.456897,"""Tiger"" William Dunlop",Requested move,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,7
9,"""V"" Is for Vagina","Why is the ""i"" capitalized?",Gunmetal Angel,"""I"" is supposed to be capatalized in ""is"" in t...",It doesn't make sense and it doesn't represent...,1,9,1.000000,"""V"" Is for Vagina","Why is the ""i"" capitalized?",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8


In [31]:
cols = merged.columns.tolist()
cols

['article',
 'thread_title',
 'editor',
 'editor_talk',
 'other_talk',
 '#editor_turns',
 '#other_turns',
 'editor_score',
 'article_name',
 'thread',
 'ftopic0',
 'ftopic1',
 'ftopic2',
 'ftopic3',
 'ftopic4',
 'ftopic5',
 'ftopic6',
 'ftopic7',
 'ftopic8',
 'ftopic9',
 'ftopic10',
 'ftopic11',
 'ftopic12',
 'ftopic13',
 'ftopic14',
 'ftopic15',
 'ftopic16',
 'ftopic17',
 'ftopic18',
 'ftopic19',
 'fold']

In [34]:
merged = merged[['fold', 'article', 'thread'] + cols[cols.index('editor'):cols.index('article_name')] + \
                cols[cols.index('ftopic0'):cols.index('fold')]]
# merged.drop(['article_name', 'thread_title'], axis=1, inplace=True)
merged

,fold,article,thread,editor,editor_talk,other_talk,#editor_turns,#other_turns,editor_score,ftopic0,...,ftopic10,ftopic11,ftopic12,ftopic13,ftopic14,ftopic15,ftopic16,ftopic17,ftopic18,ftopic19
0,7,"""Tiger"" William Dunlop",Current image,Deconstructhis,I've noticed that the current image that's acc...,I removed the deletion tag because the image i...,2,3,0.033058,2.0,...,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0
1,7,"""Tiger"" William Dunlop",Current image,Skinsmoke,::Good move ukexpat. That was really helpful (...,I've noticed that the current image that's acc...,1,4,0.482759,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,7,"""Tiger"" William Dunlop",Current image,Ukexpat,I removed the deletion tag because the image i...,I've noticed that the current image that's acc...,1,4,0.392157,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,7,"""Tiger"" William Dunlop",Requested move,CJ3370,"William ""Tiger"" Dunlop → William ""Tiger"" Dunlo...",The following discussion is an archived discus...,2,6,0.826415,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,7,"""Tiger"" William Dunlop",Requested move,CJ3370,"William ""Tiger"" Dunlop → William ""Tiger"" Dunlo...",The following discussion is an archived discus...,2,6,0.826415,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
5,7,"""Tiger"" William Dunlop",Requested move,Deconstructhis,Not a problem to merge if that is the most app...,The following discussion is an archived discus...,1,7,0.033058,2.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
6,7,"""Tiger"" William Dunlop",Requested move,Deconstructhis,Not a problem to merge if that is the most app...,The following discussion is an archived discus...,1,7,0.033058,2.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
7,7,"""Tiger"" William Dunlop",Requested move,Skinsmoke,User:CJ3370/William_Tiger_Dunlop → William Tig...,The following discussion is an archived discus...,3,5,0.456897,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
8,7,"""Tiger"" William Dunlop",Requested move,Skinsmoke,User:CJ3370/William_Tiger_Dunlop → William Tig...,The following discussion is an archived discus...,3,5,0.456897,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
9,8,"""V"" Is for Vagina","Why is the ""i"" capitalized?",Gunmetal Angel,"""I"" is supposed to be capatalized in ""is"" in t...",It doesn't make sense and it doesn't represent...,1,9,1.000000,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [35]:
len(merged)
# Need to then remove singleton conversations

64848

In [38]:
merged.to_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_allfeats.csv', index=False)

# Check for any pages with only singleton conversations

In [4]:
no_singletons = set(pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_allfeats.csv')['article'])
print(len(no_singletons))

singletons = set(pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores.csv')['article'])
print(len(singletons))

6938
10379


In [5]:
# Load fold info
with open('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/ijcnlp-rawfolds.pickle', 'rb') as f:
    folds = pickle.load(f, encoding='latin1')
    
folds

{'Ocean Park Cable Car': 0,
 'Ipswich, Queensland': 1,
 'West End, Vancouver': 2,
 'List of FLCL characters': 8,
 'Karkadann': 3,
 'List of Red Garden characters': 4,
 'List of religious leaders in 1847': 5,
 'List of whisky brands': 7,
 'John the Fearless': 6,
 'Grow a Pear': 0,
 'Architectural structure': 0,
 'Sacd': 7,
 'Flavio Costantini': 7,
 'Fear Itself (The 4400)': 8,
 'Polizei-Bataillon 33': 2,
 'Merlin': 6,
 'Holy Wars (film)': 9,
 'Rhodes University': 0,
 'Deerfoot Trail': 1,
 'List of actors who have played the Doctor': 8,
 "Multnomah County Sheriff's Office Search and Rescue": 8,
 'Mandela, Massachusetts': 9,
 'Moray Bridge': 8,
 'Attalla, Alabama': 3,
 'List of Will & Grace episodes': 3,
 'List of IARC Group 2B carcinogens': 0,
 'Peter Edwards (artist)': 5,
 'Help (movie)': 1,
 'Goodie Mob': 7,
 'Trombone Shorty': 8,
 'Halo ce': 2,
 'Insert key': 1,
 'American Idol': 3,
 '???? (Nintendo character)': 0,
 'Jimmie Angel': 0,
 'Richard Rodgers': 0,
 'Mighty Mike McGee': 1,
 '

In [12]:
# Load fold info
with open('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/ijcnlp-rawfolds.pickle', 'rb') as f:
    folds = pickle.load(f, fix_imports)
    
folds

TypeError: 'protocol' is an invalid keyword argument for this function

In [6]:
len(folds)

7211

In [7]:
fold_articles = set(folds.keys())
intersect = fold_articles.intersection(singletons)
len(intersect)

6940

In [8]:
fold_articles = set(folds.keys())
intersect = fold_articles.intersection(no_singletons)
len(intersect)

6938

In [9]:
nomatch = [art for art in fold_articles if not art in singletons]
nomatch

['Mauritaniaâ\x80\x93Senegal Border War',
 'Garda SÃ\xadochÃ¡na Ombudsman Commission',
 'Spartacus â\x80\x93 and the Sun Beneath the Sea',
 'Saxifraga Ã\x97 urbium',
 'ManezÃªs',
 'Monastery of the Patriarchate of PeÄ\x87',
 'Yaya TourÃ©',
 'MÃ\xadrzÃ¡ `AbbÃ¡s NÃºrÃ\xad',
 'DurrÃ«sâ\x80\x93TiranÃ« railway',
 'CoÅ\x9fkun Sabah',
 'A Sides Win: Singles 1992â\x80\x932005',
 "Kurt GÃ¶del's Incompleteness Theorem",
 'NapolÃ©on, Prince Imperial',
 '2008â\x80\x9309 Screen Actors Guild labor dispute',
 'Palestine 1917â\x80\x9318',
 'JosÃ© GonzÃ¡lez (singer)',
 'JosÃ© Rizal',
 'Timeline of the Syrian Civil War (Mayâ\x80\x93August 2011)',
 'Millerâ\x80\x93Rabin primality test',
 'FrantiÅ¡ek PtÃ¡Ä\x8dek',
 'EugÃ¨ne Delacroix',
 'Naruto ShippÅ«den: Saikyou Ninja Daikesshu 5',
 'Louis de FunÃ¨s',
 'Dorian Ã\x87ollaku',
 '2011â\x80\x9312 Kurdish protests in Turkey',
 'PokÃ©mon Master',
 'Branimir Å\xa0tuliÄ\x87',
 'JÃ¶rg Haider',
 'VerriÃ¨res Ridge',
 'Greeceâ\x80\x93Thailand relations',
 'Greatest 

In [10]:
len(nomatch)

271

In [11]:
search = 'Mauritania'
[art for art in singletons if search in art]

['Mauritania–Senegal Border War']

In [17]:
for art in nomatch:
    print(art)

MauritaniaâSenegal Border War
Garda SÃ­ochÃ¡na Ombudsman Commission
Spartacus â and the Sun Beneath the Sea
Saxifraga Ã urbium
ManezÃªs
Monastery of the Patriarchate of PeÄ
Yaya TourÃ©
MÃ­rzÃ¡ `AbbÃ¡s NÃºrÃ­
DurrÃ«sâTiranÃ« railway
CoÅkun Sabah
A Sides Win: Singles 1992â2005
Kurt GÃ¶del's Incompleteness Theorem
NapolÃ©on, Prince Imperial
2008â09 Screen Actors Guild labor dispute
Palestine 1917â18
JosÃ© GonzÃ¡lez (singer)
JosÃ© Rizal
Timeline of the Syrian Civil War (MayâAugust 2011)
MillerâRabin primality test
FrantiÅ¡ek PtÃ¡Äek
EugÃ¨ne Delacroix
Naruto ShippÅ«den: Saikyou Ninja Daikesshu 5
Louis de FunÃ¨s
Dorian Ãollaku
2011â12 Kurdish protests in Turkey
PokÃ©mon Master
Branimir Å tuliÄ
JÃ¶rg Haider
VerriÃ¨res Ridge
GreeceâThailand relations
Greatest Hits â HIStory Volume I
DÅ¾eko
History of the University of WisconsinâMilwaukee
Frente Popular para la LiberaciÃ³n de Saguia el-Hamra y RÃ­o de Oro
Ãthelberht of Wessex
Sanjak of VuÄitrn
Antal DorÃ¡ti
2011â

# Remove singleton conversations from features

In [39]:
feats = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_allfeats.csv')
print(len(feats))

64848


In [41]:
gped = feats.groupby(['article', 'thread'])
thread_sizes = gped.size()

thread_sizes.values >= 2

twoplus_threads = thread_sizes[thread_sizes.values >= 2].index.tolist()
twoplus_threads

[('"Tiger" William Dunlop', 'Current image'),
 ('"Tiger" William Dunlop', 'Requested move'),
 ('"V" Is for Vagina', 'Why is the "i" capitalized?'),
 ('&', '"O" as a word'),
 ('&', 'And per se and as a letter'),
 ('&', 'Evolution'),
 ('&', 'How common the misconception'),
 ('&', 'Image'),
 ('&', 'My use'),
 ("'Abd al-Ilah", 'Mixed Up?'),
 ("'Ndrangheta", 'Dead link 10'),
 ("'Ndrangheta", 'Dead link 11'),
 ("'Ndrangheta", 'Dead link 2'),
 ("'Ndrangheta", 'Dead link 3'),
 ("'Ndrangheta", 'Dead link 4'),
 ("'Ndrangheta", 'Dead link 5'),
 ("'Ndrangheta", 'Dead link 6'),
 ("'Ndrangheta", 'Dead link 7'),
 ("'Ndrangheta", 'Dead link 8'),
 ("'Ndrangheta", 'Dead link 9'),
 ("'Ndrangheta", 'NPOV template'),
 ("(I Can't Get No) Satisfaction", 'Britney Spears cover'),
 ("(I Can't Get No) Satisfaction", 'Covers and esp. singles'),
 ('-ana', 'Origin'),
 ('.hack//Outbreak', 'Lead image'),
 ('.hack//Outbreak', 'Removed text'),
 ('.hack//Outbreak', 'Suspicious links'),
 ('.nes', 'Is this a Nintendo offi

In [42]:
# Filter conversations
mask = list(map(lambda tup: tup in twoplus_threads, zip(feats['article'], feats['thread'])))
filtered = feats[mask]
len(filtered)

56528

In [43]:
filtered.to_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores_allfeats.csv', index=False)